In [1]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r "/content/drive/MyDrive/Colab Notebooks/training" "/content/training"
!cp -r "/content/drive/MyDrive/Colab Notebooks/test" "/content/test"

Mounted at /content/drive


##install

In [2]:
!pip install --upgrade batchgenerators
!pip install monai
!pip install timm
!pip install thop
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 7.5 MB/s eta 0:00:00
  Created wheel for batchgenerators: filename=batchgenerators-0.25-py3-none-any.whl size=89008 sha256=5fddd5f636f385aef098bc5558c6b984b1def6c30e9d26e4287891d58d20db19
  Stored in directory: /root/.cache/pip/wheels/9e/b0/1b/40912fb58eb167b86cbc444ddb2e6ba382b248215295f932e2
Successfully built batchgenerators


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

##load train data



In [3]:
import os
import copy
import cv2
import numpy as np
from PIL import Image, ImageTk
import torch
import PIL
from torchvision import transforms
#import config.config as cfg

def get_files(file_dir, file_list, type_str):

    for file_ in os.listdir(file_dir):
        path = os.path.join(file_dir, file_)
        if os.path.isdir(path):
            get_files(file_dir, file_list, type_str)
        else:
            if file_.rfind(type_str) !=-1:
                file_list.append(path)

def AddGaussNoise(src,sigma):
    mean = 0
    # 获取图片的高度和宽度
    height, width, channels = src.shape[0:3]
    gauss = np.random.normal(mean,sigma,(height,width,channels))


    noisy_img = src + gauss
    noisy_img = np.clip(noisy_img, 0, 1)


    # cv2.namedWindow("data2D", cv2.WINDOW_NORMAL)
    # cv2.imshow("data2D", noisy_img)
    #
    # cv2.waitKey(0)
    return noisy_img

class TrainData():
    def __init__(self, file_root, label_path, train_flag = False):
        self.data_dir = file_root
        self.label_path = label_path
        self.train_flag = train_flag

        self.train_list = None
        self.prepare(self.data_dir)


        self.transformsRotate  = transforms.RandomRotation(degrees=[-45, 45], interpolation=PIL.Image.BILINEAR)
        self.transformGray = transforms.RandomGrayscale(p=0.5)

        self.transformVFlip = transforms.RandomVerticalFlip(p=0.5)
        self.transformHFlip = transforms.RandomHorizontalFlip(p=0.5)
        # self.clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

        # cl1 = clahe.apply(img)

    def prepare(self, file_path):

        file_list = []
        get_files(file_path, file_list, "tif") #drive
        # get_files(file_path, file_list, "jpg") #CHASEDB1
        # get_files(file_path, file_list, "ppm") #STAREdatabase
        # get_files(file_path, file_list, "im.npy")#HRFdatas

        self.train_list = file_list

    def __len__(self):
        return len(self.train_list)

    def __getitem__(self, item):

        file_path = self.train_list[item]

        label_path = self.label_path + os.path.split(file_path)[-1].replace("_training.tif", "_manual1.gif")
        # label_path = self.label_path + os.path.split(file_path)[-1].replace(".jpg", "_1stHO.png")
        #label_path = self.label_path + os.path.split(file_path)[-1].replace(".ppm", ".ah.ppm")
        # label_path = self.label_path + os.path.split(file_path)[-1].replace("im.npy", "label.npy")


        img = cv2.imread(file_path)
        label = Image.open(label_path)#cv2.imread(label_path, 0)#
        label = np.array(label)
        # img = np.load(file_path)
        # label = np.load(label_path)

        data2D = img
        label2D = label

        data2D = (data2D - np.min(data2D))/(np.max(data2D) - np.min(data2D))
        label2D = label2D / np.max(label2D)


        data2D = torch.tensor(data2D).permute(2, 0, 1)
        label2D = torch.tensor(label2D)

        label2D = torch.unsqueeze(label2D, dim=0)
        mdata = torch.cat([label2D, data2D], dim=0)


        mdata = self.transformVFlip(mdata)
        mdata = self.transformHFlip(mdata)
        #random roatte
        flag = np.random.randint(0, 2, 1)[0]
        # if flag==1:
        mdata = self.transformsRotate(mdata)

        label2D = mdata[0,:,:]#.numpy()
        data2D = mdata[1:,:,:]#.permute(1, 2, 0).numpy()



        scale = np.random.randint(8, 14, 1)[0] / 10
        #
        data2D = torch.pow(data2D, scale)

        weight_ = cv2.GaussianBlur(label2D.numpy(), (5, 5), 1)

        #to Gray
        # data2D = self.transformGray(data2D)
        # data2D_ = data2D.permute(1, 2, 0).numpy()
        # label2D_ = label2D.numpy()
        # cv2.namedWindow("data2D", cv2.WINDOW_NORMAL)
        # cv2.imshow("data2D", data2D_)
        # cv2.namedWindow("label2D", cv2.WINDOW_NORMAL)
        # cv2.imshow("label2D", weight_)
        #
        # cv2.waitKey(0)



        return data2D, label2D, weight_


def data_crop(train_data, train_label, weight_):

    bn, c, height, width = train_data.shape
    CROP_SIZE = 192
    rangh = height - CROP_SIZE-1
    rangw = width - CROP_SIZE-1


    nums = 4
    train_data_ = torch.zeros((nums*bn, 3,CROP_SIZE, CROP_SIZE))
    train_label_ = torch.zeros((nums*bn,CROP_SIZE, CROP_SIZE))
    train_weight_ = torch.zeros((nums*bn,CROP_SIZE, CROP_SIZE))


    for b in range(bn):
        for i in range(nums):
            offh = np.random.randint(0, rangh, 1)[0]
            offw = np.random.randint(0, rangw, 1)[0]
            # print(height, " ", width, " ",  offh+cfg.CROP_SIZE, "   ", offw+cfg.CROP_SIZE)
            train_data_[nums*b+i,:, :, :] = train_data[b, :, offh:offh+CROP_SIZE, offw:offw+CROP_SIZE]
            train_label_[nums*b+i, :, :] = train_label[b, offh:offh+CROP_SIZE, offw:offw+CROP_SIZE]
            train_weight_[nums*b+i, :, :] = weight_[b, offh:offh+CROP_SIZE, offw:offw+CROP_SIZE]
            # data2D = train_data[0, :, offh:offh+cfg.CROP_SIZE, offw:offw+cfg.CROP_SIZE].permute(1, 2, 0).numpy()
            # label2D = train_label[0, offh:offh+cfg.CROP_SIZE, offw:offw+cfg.CROP_SIZE].numpy()
            # cv2.namedWindow("data2D", cv2.WINDOW_NORMAL)
            # cv2.imshow("data2D", data2D)
            # cv2.namedWindow("label2D", cv2.WINDOW_NORMAL)
            # cv2.imshow("label2D", label2D)
            #
        # cv2.waitKey(0)

    return train_data_, train_label_, train_weight_










##load test data

In [4]:
import os
import copy
import cv2
import numpy as np
from PIL import Image, ImageTk
import torch
import PIL
from torchvision import transforms
#import config.config as cfg

def get_files(file_dir, file_list, type_str):

    for file_ in os.listdir(file_dir):
        path = os.path.join(file_dir, file_)
        if os.path.isdir(path):
            get_files(file_dir, file_list, type_str)
        else:
            if file_.rfind(type_str) !=-1:
                file_list.append(path)


class TestData():
    def __init__(self, file_root, label_path, train_flag = False):
        self.data_dir = file_root
        self.label_path = label_path
        self.train_flag = train_flag

        self.train_list = None
        self.prepare(self.data_dir)
        # self.clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    def prepare(self, file_path):

        file_list = []
        get_files(file_path, file_list, "tif") #drive
        # get_files(file_path, file_list, "jpg") #CHASEDB1
        # get_files(file_path, file_list, "ppm") #STAREdatabase
        # get_files(file_path, file_list, "im.npy")#HRFdatas

        self.train_list = file_list

    def __len__(self):
        return len(self.train_list)

    def __getitem__(self, item):

        file_path = self.train_list[item]

        label_path = self.label_path + os.path.split(file_path)[-1].replace("_test.tif", "_manual1.gif")
        # label_path = self.label_path + os.path.split(file_path)[-1].replace(".jpg", "_1stHO.png")
        #label_path = self.label_path + os.path.split(file_path)[-1].replace(".ppm", ".ah.ppm")
        # label_path = self.label_path + os.path.split(file_path)[-1].replace("im.npy", "label.npy")


        img = cv2.imread(file_path)
        label = Image.open(label_path)#cv2.imread(label_path, 0)#
        label = np.array(label)
        # img = np.load(file_path)
        # label = np.load(label_path)


        data2D = img
        label2D = label


        data2D = (data2D - np.min(data2D)) / (np.max(data2D) - np.min(data2D))
        label2D = label2D / np.max(label2D)


        data2D = torch.tensor(data2D).permute(2, 0, 1)
        label2D = torch.tensor(label2D)


        # cv2.namedWindow("data2D", cv2.WINDOW_NORMAL)
        # cv2.imshow("data2D", data2D[0, :, :].numpy())
        # cv2.namedWindow("label2D", cv2.WINDOW_NORMAL)
        # cv2.imshow("label2D", label2D)
        #
        # cv2.waitKey(0)



        return data2D, label2D




##network_architecture

####neural network

In [5]:
#    Copyright 2020 Division of Medical Image Computing, German Cancer Research Center (DKFZ), Heidelberg, Germany
#
#    Licensed under the Apache License, Version 2.0 (the "License");
#    you may not use this file except in compliance with the License.
#    You may obtain a copy of the License at
#
#        http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.


class no_op(object):
    def __enter__(self):
        pass

    def __exit__(self, *args):
        pass

In [6]:
#    Copyright 2020 Division of Medical Image Computing, German Cancer Research Center (DKFZ), Heidelberg, Germany
#
#    Licensed under the Apache License, Version 2.0 (the "License");
#    you may not use this file except in compliance with the License.
#    You may obtain a copy of the License at
#
#        http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.

import torch


def maybe_to_torch(d):
    if isinstance(d, list):
        d = [maybe_to_torch(i) if not isinstance(i, torch.Tensor) else i for i in d]
    elif not isinstance(d, torch.Tensor):
        d = torch.from_numpy(d).float()
    return d


def to_cuda(data, non_blocking=True, gpu_id=0):
    if isinstance(data, list):
        data = [i.cuda(gpu_id, non_blocking=non_blocking) for i in data]
    else:
        data = data.cuda(gpu_id, non_blocking=non_blocking)
    return data

In [7]:
#    Copyright 2020 Division of Medical Image Computing, German Cancer Research Center (DKFZ), Heidelberg, Germany
#
#    Licensed under the Apache License, Version 2.0 (the "License");
#    you may not use this file except in compliance with the License.
#    You may obtain a copy of the License at
#
#        http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.


import numpy as np
from batchgenerators.augmentations.utils import pad_nd_image
#from utilities.random_stuff import no_op
#from utilities.to_torch import to_cuda, maybe_to_torch
from torch import nn
import torch
from scipy.ndimage.filters import gaussian_filter
from typing import Union, Tuple, List

from torch.cuda.amp import autocast


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

    def get_device(self):
        if next(self.parameters()).device == "cpu":
            return "cpu"
        else:
            return next(self.parameters()).device.index

    def set_device(self, device):
        if device == "cpu":
            self.cpu()
        else:
            self.cuda(device)

    def forward(self, x):
        raise NotImplementedError


class SegmentationNetwork(NeuralNetwork):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        # if we have 5 pooling then our patch size must be divisible by 2**5
        self.input_shape_must_be_divisible_by = None  # for example in a 2d network that does 5 pool in x and 6 pool
        # in y this would be (32, 64)

        # we need to know this because we need to know if we are a 2d or a 3d netowrk
        self.conv_op = None  # nn.Conv2d or nn.Conv3d

        # this tells us how many channely we have in the output. Important for preallocation in inference
        self.num_classes = None  # number of channels in the output

        # depending on the loss, we do not hard code a nonlinearity into the architecture. To aggregate predictions
        # during inference, we need to apply the nonlinearity, however. So it is important to let the newtork know what
        # to apply in inference. For the most part this will be softmax
        self.inference_apply_nonlin = lambda x: x  # softmax_helper

        # This is for saving a gaussian importance map for inference. It weights voxels higher that are closer to the
        # center. Prediction at the borders are often less accurate and are thus downweighted. Creating these Gaussians
        # can be expensive, so it makes sense to save and reuse them.
        self._gaussian_3d = self._patch_size_for_gaussian_3d = None
        self._gaussian_2d = self._patch_size_for_gaussian_2d = None

    def predict_3D(self, x: np.ndarray, do_mirroring: bool, mirror_axes: Tuple[int, ...] = (0, 1, 2),
                   use_sliding_window: bool = False,
                   step_size: float = 0.5, patch_size: Tuple[int, ...] = None, regions_class_order: Tuple[int, ...] = None,
                   use_gaussian: bool = False, pad_border_mode: str = "constant",
                   pad_kwargs: dict = None, all_in_gpu: bool = False,
                   verbose: bool = True, mixed_precision: bool = True) -> Tuple[np.ndarray, np.ndarray]:
        """
        Use this function to predict a 3D image. It does not matter whether the network is a 2D or 3D U-Net, it will
        detect that automatically and run the appropriate code.

        When running predictions, you need to specify whether you want to run fully convolutional of sliding window
        based inference. We very strongly recommend you use sliding window with the default settings.

        It is the responsibility of the user to make sure the network is in the proper mode (eval for inference!). If
        the network is not in eval mode it will print a warning.

        :param x: Your input data. Must be a nd.ndarray of shape (c, x, y, z).
        :param do_mirroring: If True, use test time data augmentation in the form of mirroring
        :param mirror_axes: Determines which axes to use for mirroing. Per default, mirroring is done along all three
        axes
        :param use_sliding_window: if True, run sliding window prediction. Heavily recommended! This is also the default
        :param step_size: When running sliding window prediction, the step size determines the distance between adjacent
        predictions. The smaller the step size, the denser the predictions (and the longer it takes!). Step size is given
        as a fraction of the patch_size. 0.5 is the default and means that wen advance by patch_size * 0.5 between
        predictions. step_size cannot be larger than 1!
        :param patch_size: The patch size that was used for training the network. Do not use different patch sizes here,
        this will either crash or give potentially less accurate segmentations
        :param regions_class_order: Fabian only
        :param use_gaussian: (Only applies to sliding window prediction) If True, uses a Gaussian importance weighting
         to weigh predictions closer to the center of the current patch higher than those at the borders. The reason
         behind this is that the segmentation accuracy decreases towards the borders. Default (and recommended): True
        :param pad_border_mode: leave this alone
        :param pad_kwargs: leave this alone
        :param all_in_gpu: experimental. You probably want to leave this as is it
        :param verbose: Do you want a wall of text? If yes then set this to True
        :param mixed_precision: if True, will run inference in mixed precision with autocast()
        :return:
        """
        torch.cuda.empty_cache()

        assert step_size <= 1, 'step_size must be smaller than 1. Otherwise there will be a gap between consecutive ' \
                               'predictions'

        if verbose: print("debug: mirroring", do_mirroring, "mirror_axes", mirror_axes)

        assert self.get_device() != "cpu", "CPU not implemented"

        if pad_kwargs is None:
            pad_kwargs = {'constant_values': 0}

        # A very long time ago the mirror axes were (2, 3, 4) for a 3d network. This is just to intercept any old
        # code that uses this convention
        self.conv_op = nn.Conv3d
        if len(mirror_axes):
            if self.conv_op == nn.Conv2d:
                if max(mirror_axes) > 1:
                    raise ValueError("mirror axes. duh")
            if self.conv_op == nn.Conv3d:
                if max(mirror_axes) > 2:
                    raise ValueError("mirror axes. duh")

        if self.training:
            print('WARNING! Network is in train mode during inference. This may be intended, or not...')

        assert len(x.shape) == 4, "data must have shape (c,x,y,z)"

        if mixed_precision:
            context = autocast
        else:
            context = no_op

        with context():
            with torch.no_grad():
                if self.conv_op == nn.Conv3d:
                    if use_sliding_window:
                        res = self._internal_predict_3D_3Dconv_tiled(x, step_size, do_mirroring, mirror_axes, patch_size,
                                                                     regions_class_order, use_gaussian, pad_border_mode,
                                                                     pad_kwargs=pad_kwargs, all_in_gpu=all_in_gpu,
                                                                     verbose=verbose)
                    else:
                        res = self._internal_predict_3D_3Dconv(x, patch_size, do_mirroring, mirror_axes, regions_class_order,
                                                               pad_border_mode, pad_kwargs=pad_kwargs, verbose=verbose)
                elif self.conv_op == nn.Conv2d:
                    if use_sliding_window:
                        res = self._internal_predict_3D_2Dconv_tiled(x, patch_size, do_mirroring, mirror_axes, step_size,
                                                                     regions_class_order, use_gaussian, pad_border_mode,
                                                                     pad_kwargs, all_in_gpu, False)
                    else:
                        res = self._internal_predict_3D_2Dconv(x, patch_size, do_mirroring, mirror_axes, regions_class_order,
                                                               pad_border_mode, pad_kwargs, all_in_gpu, False)
                else:
                    raise RuntimeError("Invalid conv op, cannot determine what dimensionality (2d/3d) the network is")

        return res

    def predict_2D(self, x, do_mirroring: bool, mirror_axes: tuple = (0, 1, 2), use_sliding_window: bool = False,
                   step_size: float = 0.5, patch_size: tuple = None, regions_class_order: tuple = None,
                   use_gaussian: bool = False, pad_border_mode: str = "constant",
                   pad_kwargs: dict = None, all_in_gpu: bool = False,
                   verbose: bool = True, mixed_precision: bool = True) -> Tuple[np.ndarray, np.ndarray]:
        """
        Use this function to predict a 2D image. If this is a 3D U-Net it will crash because you cannot predict a 2D
        image with that (you dummy).

        When running predictions, you need to specify whether you want to run fully convolutional of sliding window
        based inference. We very strongly recommend you use sliding window with the default settings.

        It is the responsibility of the user to make sure the network is in the proper mode (eval for inference!). If
        the network is not in eval mode it will print a warning.

        :param x: Your input data. Must be a nd.ndarray of shape (c, x, y).
        :param do_mirroring: If True, use test time data augmentation in the form of mirroring
        :param mirror_axes: Determines which axes to use for mirroing. Per default, mirroring is done along all three
        axes
        :param use_sliding_window: if True, run sliding window prediction. Heavily recommended! This is also the default
        :param step_size: When running sliding window prediction, the step size determines the distance between adjacent
        predictions. The smaller the step size, the denser the predictions (and the longer it takes!). Step size is given
        as a fraction of the patch_size. 0.5 is the default and means that wen advance by patch_size * 0.5 between
        predictions. step_size cannot be larger than 1!
        :param patch_size: The patch size that was used for training the network. Do not use different patch sizes here,
        this will either crash or give potentially less accurate segmentations
        :param regions_class_order: Fabian only
        :param use_gaussian: (Only applies to sliding window prediction) If True, uses a Gaussian importance weighting
         to weigh predictions closer to the center of the current patch higher than those at the borders. The reason
         behind this is that the segmentation accuracy decreases towards the borders. Default (and recommended): True
        :param pad_border_mode: leave this alone
        :param pad_kwargs: leave this alone
        :param all_in_gpu: experimental. You probably want to leave this as is it
        :param verbose: Do you want a wall of text? If yes then set this to True
        :return:
        """
        torch.cuda.empty_cache()

        assert step_size <= 1, 'step_size must be smaler than 1. Otherwise there will be a gap between consecutive ' \
                               'predictions'

        if self.conv_op == nn.Conv3d:
            raise RuntimeError("Cannot predict 2d if the network is 3d. Dummy.")

        if verbose: print("debug: mirroring", do_mirroring, "mirror_axes", mirror_axes)

        assert self.get_device() != "cpu", "CPU not implemented"

        if pad_kwargs is None:
            pad_kwargs = {'constant_values': 0}

        # A very long time ago the mirror axes were (2, 3) for a 2d network. This is just to intercept any old
        # code that uses this convention
        if len(mirror_axes):
            if max(mirror_axes) > 1:
                raise ValueError("mirror axes. duh")

        if self.training:
            print('WARNING! Network is in train mode during inference. This may be intended, or not...')

        assert len(x.shape) == 3, "data must have shape (c,x,y)"

        if mixed_precision:
            context = autocast
        else:
            context = no_op

        with context():
            with torch.no_grad():
                if self.conv_op == nn.Conv2d:
                    if use_sliding_window:
                        res = self._internal_predict_2D_2Dconv_tiled(x, step_size, do_mirroring, mirror_axes, patch_size,
                                                                     regions_class_order, use_gaussian, pad_border_mode,
                                                                     pad_kwargs, all_in_gpu, verbose)
                    else:
                        res = self._internal_predict_2D_2Dconv(x, patch_size, do_mirroring, mirror_axes, regions_class_order,
                                                               pad_border_mode, pad_kwargs, verbose)
                else:
                    raise RuntimeError("Invalid conv op, cannot determine what dimensionality (2d/3d) the network is")

        return res

    @staticmethod
    def _get_gaussian(patch_size, sigma_scale=1. / 8) -> np.ndarray:
        tmp = np.zeros(patch_size)
        center_coords = [i // 2 for i in patch_size]
        sigmas = [i * sigma_scale for i in patch_size]
        tmp[tuple(center_coords)] = 1
        gaussian_importance_map = gaussian_filter(tmp, sigmas, 0, mode='constant', cval=0)
        gaussian_importance_map = gaussian_importance_map / np.max(gaussian_importance_map) * 1
        gaussian_importance_map = gaussian_importance_map.astype(np.float32)

        # gaussian_importance_map cannot be 0, otherwise we may end up with nans!
        gaussian_importance_map[gaussian_importance_map == 0] = np.min(
            gaussian_importance_map[gaussian_importance_map != 0])

        return gaussian_importance_map

    @staticmethod
    def _compute_steps_for_sliding_window(patch_size: Tuple[int, ...], image_size: Tuple[int, ...], step_size: float) -> List[List[int]]:
        assert [i >= j for i, j in zip(image_size, patch_size)], "image size must be as large or larger than patch_size"
        assert 0 < step_size <= 1, 'step_size must be larger than 0 and smaller or equal to 1'

        # our step width is patch_size*step_size at most, but can be narrower. For example if we have image size of
        # 110, patch size of 64 and step_size of 0.5, then we want to make 3 steps starting at coordinate 0, 23, 46
        target_step_sizes_in_voxels = [i * step_size for i in patch_size]

        num_steps = [int(np.ceil((i - k) / j)) + 1 for i, j, k in zip(image_size, target_step_sizes_in_voxels, patch_size)]

        steps = []
        for dim in range(len(patch_size)):
            # the highest step value for this dimension is
            max_step_value = image_size[dim] - patch_size[dim]
            if num_steps[dim] > 1:
                actual_step_size = max_step_value / (num_steps[dim] - 1)
            else:
                actual_step_size = 99999999999  # does not matter because there is only one step at 0

            steps_here = [int(np.round(actual_step_size * i)) for i in range(num_steps[dim])]

            steps.append(steps_here)

        return steps

    def _internal_predict_3D_3Dconv_tiled(self, x: np.ndarray, step_size: float, do_mirroring: bool, mirror_axes: tuple,
                                          patch_size: tuple, regions_class_order: tuple, use_gaussian: bool,
                                          pad_border_mode: str, pad_kwargs: dict, all_in_gpu: bool,
                                          verbose: bool) -> Tuple[np.ndarray, np.ndarray]:

        # better safe than sorry
        assert len(x.shape) == 4, "x must be (c, x, y, z)"
        assert self.get_device() != "cpu"
        if verbose: print("step_size:", step_size)
        if verbose: print("do mirror:", do_mirroring)

        assert patch_size is not None, "patch_size cannot be None for tiled prediction"

        # for sliding window inference the image must at least be as large as the patch size. It does not matter
        # whether the shape is divisible by 2**num_pool as long as the patch size is
        data, slicer = pad_nd_image(x, patch_size, pad_border_mode, pad_kwargs, True, None)
        data_shape = data.shape  # still c, x, y, z

        # compute the steps for sliding window
        steps = self._compute_steps_for_sliding_window(patch_size, data_shape[1:], step_size)
        num_tiles = len(steps[0]) * len(steps[1]) * len(steps[2])

        if verbose:
            print("data shape:", data_shape)
            print("patch size:", patch_size)
            print("steps (x, y, and z):", steps)
            print("number of tiles:", num_tiles)

        # we only need to compute that once. It can take a while to compute this due to the large sigma in
        # gaussian_filter
        if use_gaussian and num_tiles > 1:
            if self._gaussian_3d is None or not all(
                    [i == j for i, j in zip(patch_size, self._patch_size_for_gaussian_3d)]):
                if verbose: print('computing Gaussian')
                gaussian_importance_map = self._get_gaussian(patch_size, sigma_scale=1. / 8)

                self._gaussian_3d = gaussian_importance_map
                self._patch_size_for_gaussian_3d = patch_size
            else:
                if verbose: print("using precomputed Gaussian")
                gaussian_importance_map = self._gaussian_3d

            gaussian_importance_map = torch.from_numpy(gaussian_importance_map).cuda(self.get_device(),
                                                                                     non_blocking=True)

        else:
            gaussian_importance_map = None

        if all_in_gpu:
            # If we run the inference in GPU only (meaning all tensors are allocated on the GPU, this reduces
            # CPU-GPU communication but required more GPU memory) we need to preallocate a few things on GPU

            if use_gaussian and num_tiles > 1:
                # half precision for the outputs should be good enough. If the outputs here are half, the
                # gaussian_importance_map should be as well
                gaussian_importance_map = gaussian_importance_map.half()

                # make sure we did not round anything to 0
                gaussian_importance_map[gaussian_importance_map == 0] = gaussian_importance_map[
                    gaussian_importance_map != 0].min()

                add_for_nb_of_preds = gaussian_importance_map
            else:
                add_for_nb_of_preds = torch.ones(data.shape[1:], device=self.get_device())

            if verbose: print("initializing result array (on GPU)")
            aggregated_results = torch.zeros([self.num_classes] + list(data.shape[1:]), dtype=torch.half,
                                             device=self.get_device())

            if verbose: print("moving data to GPU")
            data = torch.from_numpy(data).cuda(self.get_device(), non_blocking=True)

            if verbose: print("initializing result_numsamples (on GPU)")
            aggregated_nb_of_predictions = torch.zeros([self.num_classes] + list(data.shape[1:]), dtype=torch.half,
                                                       device=self.get_device())
        else:
            if use_gaussian and num_tiles > 1:
                add_for_nb_of_preds = self._gaussian_3d
            else:
                add_for_nb_of_preds = np.ones(data.shape[1:], dtype=np.float32)

            aggregated_results = np.zeros([self.num_classes] + list(data.shape[1:]), dtype=np.float32)
            aggregated_nb_of_predictions = np.zeros([self.num_classes] + list(data.shape[1:]), dtype=np.float32)

        for x in steps[0]:
            lb_x = x
            ub_x = x + patch_size[0]
            for y in steps[1]:
                lb_y = y
                ub_y = y + patch_size[1]
                for z in steps[2]:
                    lb_z = z
                    ub_z = z + patch_size[2]

                    predicted_patch = self._internal_maybe_mirror_and_pred_3D(
                        data[None, :, lb_x:ub_x, lb_y:ub_y, lb_z:ub_z], mirror_axes, do_mirroring,
                        gaussian_importance_map)[0]

                    if all_in_gpu:
                        predicted_patch = predicted_patch.half()
                    else:
                        predicted_patch = predicted_patch.cpu().numpy()

                    aggregated_results[:, lb_x:ub_x, lb_y:ub_y, lb_z:ub_z] += predicted_patch
                    aggregated_nb_of_predictions[:, lb_x:ub_x, lb_y:ub_y, lb_z:ub_z] += add_for_nb_of_preds

        # we reverse the padding here (remeber that we padded the input to be at least as large as the patch size
        slicer = tuple(
            [slice(0, aggregated_results.shape[i]) for i in
             range(len(aggregated_results.shape) - (len(slicer) - 1))] + slicer[1:])
        aggregated_results = aggregated_results[slicer]
        aggregated_nb_of_predictions = aggregated_nb_of_predictions[slicer]

        # computing the class_probabilities by dividing the aggregated result with result_numsamples
        class_probabilities = aggregated_results / aggregated_nb_of_predictions

        if regions_class_order is None:
            predicted_segmentation = class_probabilities.argmax(0)
        else:
            if all_in_gpu:
                class_probabilities_here = class_probabilities.detach().cpu().numpy()
            else:
                class_probabilities_here = class_probabilities
            predicted_segmentation = np.zeros(class_probabilities_here.shape[1:], dtype=np.float32)
            for i, c in enumerate(regions_class_order):
                predicted_segmentation[class_probabilities_here[i] > 0.5] = c

        if all_in_gpu:
            if verbose: print("copying results to CPU")

            if regions_class_order is None:
                predicted_segmentation = predicted_segmentation.detach().cpu().numpy()

            class_probabilities = class_probabilities.detach().cpu().numpy()

        if verbose: print("prediction done")
        return predicted_segmentation, class_probabilities

    def _internal_predict_2D_2Dconv(self, x: np.ndarray, min_size: Tuple[int, int], do_mirroring: bool,
                                    mirror_axes: tuple = (0, 1, 2), regions_class_order: tuple = None,
                                    pad_border_mode: str = "constant", pad_kwargs: dict = None,
                                    verbose: bool = True) -> Tuple[np.ndarray, np.ndarray]:
        """
        This one does fully convolutional inference. No sliding window
        """
        assert len(x.shape) == 3, "x must be (c, x, y)"
        assert self.get_device() != "cpu"
        assert self.input_shape_must_be_divisible_by is not None, 'input_shape_must_be_divisible_by must be set to ' \
                                                                  'run _internal_predict_2D_2Dconv'
        if verbose: print("do mirror:", do_mirroring)

        data, slicer = pad_nd_image(x, min_size, pad_border_mode, pad_kwargs, True,
                                    self.input_shape_must_be_divisible_by)

        predicted_probabilities = self._internal_maybe_mirror_and_pred_2D(data[None], mirror_axes, do_mirroring,
                                                                          None)[0]

        slicer = tuple(
            [slice(0, predicted_probabilities.shape[i]) for i in range(len(predicted_probabilities.shape) -
                                                                       (len(slicer) - 1))] + slicer[1:])
        predicted_probabilities = predicted_probabilities[slicer]

        if regions_class_order is None:
            predicted_segmentation = predicted_probabilities.argmax(0)
            predicted_segmentation = predicted_segmentation.detach().cpu().numpy()
            predicted_probabilities = predicted_probabilities.detach().cpu().numpy()
        else:
            predicted_probabilities = predicted_probabilities.detach().cpu().numpy()
            predicted_segmentation = np.zeros(predicted_probabilities.shape[1:], dtype=np.float32)
            for i, c in enumerate(regions_class_order):
                predicted_segmentation[predicted_probabilities[i] > 0.5] = c

        return predicted_segmentation, predicted_probabilities

    def _internal_predict_3D_3Dconv(self, x: np.ndarray, min_size: Tuple[int, ...], do_mirroring: bool,
                                    mirror_axes: tuple = (0, 1, 2), regions_class_order: tuple = None,
                                    pad_border_mode: str = "constant", pad_kwargs: dict = None,
                                    verbose: bool = True) -> Tuple[np.ndarray, np.ndarray]:
        """
        This one does fully convolutional inference. No sliding window
        """
        assert len(x.shape) == 4, "x must be (c, x, y, z)"
        assert self.get_device() != "cpu"
        assert self.input_shape_must_be_divisible_by is not None, 'input_shape_must_be_divisible_by must be set to ' \
                                                                  'run _internal_predict_3D_3Dconv'
        if verbose: print("do mirror:", do_mirroring)

        data, slicer = pad_nd_image(x, min_size, pad_border_mode, pad_kwargs, True,
                                    self.input_shape_must_be_divisible_by)

        predicted_probabilities = self._internal_maybe_mirror_and_pred_3D(data[None], mirror_axes, do_mirroring,
                                                                          None)[0]

        slicer = tuple(
            [slice(0, predicted_probabilities.shape[i]) for i in range(len(predicted_probabilities.shape) -
                                                                       (len(slicer) - 1))] + slicer[1:])
        predicted_probabilities = predicted_probabilities[slicer]

        if regions_class_order is None:
            predicted_segmentation = predicted_probabilities.argmax(0)
            predicted_segmentation = predicted_segmentation.detach().cpu().numpy()
            predicted_probabilities = predicted_probabilities.detach().cpu().numpy()
        else:
            predicted_probabilities = predicted_probabilities.detach().cpu().numpy()
            predicted_segmentation = np.zeros(predicted_probabilities.shape[1:], dtype=np.float32)
            for i, c in enumerate(regions_class_order):
                predicted_segmentation[predicted_probabilities[i] > 0.5] = c

        return predicted_segmentation, predicted_probabilities

    def _internal_maybe_mirror_and_pred_3D(self, x: Union[np.ndarray, torch.tensor], mirror_axes: tuple,
                                           do_mirroring: bool = True,
                                           mult: np.ndarray or torch.tensor = None) -> torch.tensor:
        assert len(x.shape) == 5, 'x must be (b, c, x, y, z)'
        # everything in here takes place on the GPU. If x and mult are not yet on GPU this will be taken care of here
        # we now return a cuda tensor! Not numpy array!

        x = to_cuda(maybe_to_torch(x), gpu_id=self.get_device())
        result_torch = torch.zeros([1, self.num_classes] + list(x.shape[2:]),
                                   dtype=torch.float).cuda(self.get_device(), non_blocking=True)

        if mult is not None:
            mult = to_cuda(maybe_to_torch(mult), gpu_id=self.get_device())

        if do_mirroring:
            mirror_idx = 8
            num_results = 2 ** len(mirror_axes)
        else:
            mirror_idx = 1
            num_results = 1

        for m in range(mirror_idx):
            if m == 0:
                pred = self.inference_apply_nonlin(self(x))
                result_torch += 1 / num_results * pred

            if m == 1 and (2 in mirror_axes):
                pred = self.inference_apply_nonlin(self(torch.flip(x, (4, ))))
                result_torch += 1 / num_results * torch.flip(pred, (4,))

            if m == 2 and (1 in mirror_axes):
                pred = self.inference_apply_nonlin(self(torch.flip(x, (3, ))))
                result_torch += 1 / num_results * torch.flip(pred, (3,))

            if m == 3 and (2 in mirror_axes) and (1 in mirror_axes):
                pred = self.inference_apply_nonlin(self(torch.flip(x, (4, 3))))
                result_torch += 1 / num_results * torch.flip(pred, (4, 3))

            if m == 4 and (0 in mirror_axes):
                pred = self.inference_apply_nonlin(self(torch.flip(x, (2, ))))
                result_torch += 1 / num_results * torch.flip(pred, (2,))

            if m == 5 and (0 in mirror_axes) and (2 in mirror_axes):
                pred = self.inference_apply_nonlin(self(torch.flip(x, (4, 2))))
                result_torch += 1 / num_results * torch.flip(pred, (4, 2))

            if m == 6 and (0 in mirror_axes) and (1 in mirror_axes):
                pred = self.inference_apply_nonlin(self(torch.flip(x, (3, 2))))
                result_torch += 1 / num_results * torch.flip(pred, (3, 2))

            if m == 7 and (0 in mirror_axes) and (1 in mirror_axes) and (2 in mirror_axes):
                pred = self.inference_apply_nonlin(self(torch.flip(x, (4, 3, 2))))
                result_torch += 1 / num_results * torch.flip(pred, (4, 3, 2))

        if mult is not None:
            result_torch[:, :] *= mult

        return result_torch

    def _internal_maybe_mirror_and_pred_2D(self, x: Union[np.ndarray, torch.tensor], mirror_axes: tuple,
                                           do_mirroring: bool = True,
                                           mult: np.ndarray or torch.tensor = None) -> torch.tensor:
        # everything in here takes place on the GPU. If x and mult are not yet on GPU this will be taken care of here
        # we now return a cuda tensor! Not numpy array!
        assert len(x.shape) == 4, 'x must be (b, c, x, y)'

        x = to_cuda(maybe_to_torch(x), gpu_id=self.get_device())
        result_torch = torch.zeros([x.shape[0], self.num_classes] + list(x.shape[2:]),
                                   dtype=torch.float).cuda(self.get_device(), non_blocking=True)

        if mult is not None:
            mult = to_cuda(maybe_to_torch(mult), gpu_id=self.get_device())

        if do_mirroring:
            mirror_idx = 4
            num_results = 2 ** len(mirror_axes)
        else:
            mirror_idx = 1
            num_results = 1

        for m in range(mirror_idx):
            if m == 0:
                pred = self.inference_apply_nonlin(self(x))
                result_torch += 1 / num_results * pred

            if m == 1 and (1 in mirror_axes):
                pred = self.inference_apply_nonlin(self(torch.flip(x, (3, ))))
                result_torch += 1 / num_results * torch.flip(pred, (3, ))

            if m == 2 and (0 in mirror_axes):
                pred = self.inference_apply_nonlin(self(torch.flip(x, (2, ))))
                result_torch += 1 / num_results * torch.flip(pred, (2, ))

            if m == 3 and (0 in mirror_axes) and (1 in mirror_axes):
                pred = self.inference_apply_nonlin(self(torch.flip(x, (3, 2))))
                result_torch += 1 / num_results * torch.flip(pred, (3, 2))

        if mult is not None:
            result_torch[:, :] *= mult

        return result_torch

    def _internal_predict_2D_2Dconv_tiled(self, x: np.ndarray, step_size: float, do_mirroring: bool, mirror_axes: tuple,
                                          patch_size: tuple, regions_class_order: tuple, use_gaussian: bool,
                                          pad_border_mode: str, pad_kwargs: dict, all_in_gpu: bool,
                                          verbose: bool) -> Tuple[np.ndarray, np.ndarray]:
        # better safe than sorry
        assert len(x.shape) == 3, "x must be (c, x, y)"
        assert self.get_device() != "cpu"
        if verbose: print("step_size:", step_size)
        if verbose: print("do mirror:", do_mirroring)

        assert patch_size is not None, "patch_size cannot be None for tiled prediction"

        # for sliding window inference the image must at least be as large as the patch size. It does not matter
        # whether the shape is divisible by 2**num_pool as long as the patch size is
        data, slicer = pad_nd_image(x, patch_size, pad_border_mode, pad_kwargs, True, None)
        data_shape = data.shape  # still c, x, y

        # compute the steps for sliding window
        steps = self._compute_steps_for_sliding_window(patch_size, data_shape[1:], step_size)
        num_tiles = len(steps[0]) * len(steps[1])

        if verbose:
            print("data shape:", data_shape)
            print("patch size:", patch_size)
            print("steps (x, y, and z):", steps)
            print("number of tiles:", num_tiles)

        # we only need to compute that once. It can take a while to compute this due to the large sigma in
        # gaussian_filter
        if use_gaussian and num_tiles > 1:
            if self._gaussian_2d is None or not all(
                    [i == j for i, j in zip(patch_size, self._patch_size_for_gaussian_2d)]):
                if verbose: print('computing Gaussian')
                gaussian_importance_map = self._get_gaussian(patch_size, sigma_scale=1. / 8)

                self._gaussian_2d = gaussian_importance_map
                self._patch_size_for_gaussian_2d = patch_size
            else:
                if verbose: print("using precomputed Gaussian")
                gaussian_importance_map = self._gaussian_2d

            gaussian_importance_map = torch.from_numpy(gaussian_importance_map).cuda(self.get_device(),
                                                                                     non_blocking=True)
        else:
            gaussian_importance_map = None

        if all_in_gpu:
            # If we run the inference in GPU only (meaning all tensors are allocated on the GPU, this reduces
            # CPU-GPU communication but required more GPU memory) we need to preallocate a few things on GPU

            if use_gaussian and num_tiles > 1:
                # half precision for the outputs should be good enough. If the outputs here are half, the
                # gaussian_importance_map should be as well
                gaussian_importance_map = gaussian_importance_map.half()

                # make sure we did not round anything to 0
                gaussian_importance_map[gaussian_importance_map == 0] = gaussian_importance_map[
                    gaussian_importance_map != 0].min()

                add_for_nb_of_preds = gaussian_importance_map
            else:
                add_for_nb_of_preds = torch.ones(data.shape[1:], device=self.get_device())

            if verbose: print("initializing result array (on GPU)")
            aggregated_results = torch.zeros([self.num_classes] + list(data.shape[1:]), dtype=torch.half,
                                             device=self.get_device())

            if verbose: print("moving data to GPU")
            data = torch.from_numpy(data).cuda(self.get_device(), non_blocking=True)

            if verbose: print("initializing result_numsamples (on GPU)")
            aggregated_nb_of_predictions = torch.zeros([self.num_classes] + list(data.shape[1:]), dtype=torch.half,
                                                       device=self.get_device())
        else:
            if use_gaussian and num_tiles > 1:
                add_for_nb_of_preds = self._gaussian_2d
            else:
                add_for_nb_of_preds = np.ones(data.shape[1:], dtype=np.float32)
            aggregated_results = np.zeros([self.num_classes] + list(data.shape[1:]), dtype=np.float32)
            aggregated_nb_of_predictions = np.zeros([self.num_classes] + list(data.shape[1:]), dtype=np.float32)

        for x in steps[0]:
            lb_x = x
            ub_x = x + patch_size[0]
            for y in steps[1]:
                lb_y = y
                ub_y = y + patch_size[1]

                predicted_patch = self._internal_maybe_mirror_and_pred_2D(
                    data[None, :, lb_x:ub_x, lb_y:ub_y], mirror_axes, do_mirroring,
                    gaussian_importance_map)[0]

                if all_in_gpu:
                    predicted_patch = predicted_patch.half()
                else:
                    predicted_patch = predicted_patch.cpu().numpy()

                aggregated_results[:, lb_x:ub_x, lb_y:ub_y] += predicted_patch
                aggregated_nb_of_predictions[:, lb_x:ub_x, lb_y:ub_y] += add_for_nb_of_preds

        # we reverse the padding here (remeber that we padded the input to be at least as large as the patch size
        slicer = tuple(
            [slice(0, aggregated_results.shape[i]) for i in
             range(len(aggregated_results.shape) - (len(slicer) - 1))] + slicer[1:])
        aggregated_results = aggregated_results[slicer]
        aggregated_nb_of_predictions = aggregated_nb_of_predictions[slicer]

        # computing the class_probabilities by dividing the aggregated result with result_numsamples
        class_probabilities = aggregated_results / aggregated_nb_of_predictions

        if regions_class_order is None:
            predicted_segmentation = class_probabilities.argmax(0)
        else:
            if all_in_gpu:
                class_probabilities_here = class_probabilities.detach().cpu().numpy()
            else:
                class_probabilities_here = class_probabilities
            predicted_segmentation = np.zeros(class_probabilities_here.shape[1:], dtype=np.float32)
            for i, c in enumerate(regions_class_order):
                predicted_segmentation[class_probabilities_here[i] > 0.5] = c

        if all_in_gpu:
            if verbose: print("copying results to CPU")

            if regions_class_order is None:
                predicted_segmentation = predicted_segmentation.detach().cpu().numpy()

            class_probabilities = class_probabilities.detach().cpu().numpy()

        if verbose: print("prediction done")
        return predicted_segmentation, class_probabilities

    def _internal_predict_3D_2Dconv(self, x: np.ndarray, min_size: Tuple[int, int], do_mirroring: bool,
                                    mirror_axes: tuple = (0, 1), regions_class_order: tuple = None,
                                    pad_border_mode: str = "constant", pad_kwargs: dict = None,
                                    all_in_gpu: bool = False, verbose: bool = True) -> Tuple[np.ndarray, np.ndarray]:
        if all_in_gpu:
            raise NotImplementedError
        assert len(x.shape) == 4, "data must be c, x, y, z"
        predicted_segmentation = []
        softmax_pred = []
        for s in range(x.shape[1]):
            pred_seg, softmax_pres = self._internal_predict_2D_2Dconv(
                x[:, s], min_size, do_mirroring, mirror_axes, regions_class_order, pad_border_mode, pad_kwargs, verbose)
            predicted_segmentation.append(pred_seg[None])
            softmax_pred.append(softmax_pres[None])
        predicted_segmentation = np.vstack(predicted_segmentation)
        softmax_pred = np.vstack(softmax_pred).transpose((1, 0, 2, 3))
        return predicted_segmentation, softmax_pred

    def predict_3D_pseudo3D_2Dconv(self, x: np.ndarray, min_size: Tuple[int, int], do_mirroring: bool,
                                   mirror_axes: tuple = (0, 1), regions_class_order: tuple = None,
                                   pseudo3D_slices: int = 5, all_in_gpu: bool = False,
                                   pad_border_mode: str = "constant", pad_kwargs: dict = None,
                                   verbose: bool = True) -> Tuple[np.ndarray, np.ndarray]:
        if all_in_gpu:
            raise NotImplementedError
        assert len(x.shape) == 4, "data must be c, x, y, z"
        assert pseudo3D_slices % 2 == 1, "pseudo3D_slices must be odd"
        extra_slices = (pseudo3D_slices - 1) // 2

        shp_for_pad = np.array(x.shape)
        shp_for_pad[1] = extra_slices

        pad = np.zeros(shp_for_pad, dtype=np.float32)
        data = np.concatenate((pad, x, pad), 1)

        predicted_segmentation = []
        softmax_pred = []
        for s in range(extra_slices, data.shape[1] - extra_slices):
            d = data[:, (s - extra_slices):(s + extra_slices + 1)]
            d = d.reshape((-1, d.shape[-2], d.shape[-1]))
            pred_seg, softmax_pres = \
                self._internal_predict_2D_2Dconv(d, min_size, do_mirroring, mirror_axes,
                                                 regions_class_order, pad_border_mode, pad_kwargs, verbose)
            predicted_segmentation.append(pred_seg[None])
            softmax_pred.append(softmax_pres[None])
        predicted_segmentation = np.vstack(predicted_segmentation)
        softmax_pred = np.vstack(softmax_pred).transpose((1, 0, 2, 3))

        return predicted_segmentation, softmax_pred

    def _internal_predict_3D_2Dconv_tiled(self, x: np.ndarray, patch_size: Tuple[int, int], do_mirroring: bool,
                                          mirror_axes: tuple = (0, 1), step_size: float = 0.5,
                                          regions_class_order: tuple = None, use_gaussian: bool = False,
                                          pad_border_mode: str = "edge", pad_kwargs: dict =None,
                                          all_in_gpu: bool = False,
                                          verbose: bool = True) -> Tuple[np.ndarray, np.ndarray]:
        if all_in_gpu:
            raise NotImplementedError

        assert len(x.shape) == 4, "data must be c, x, y, z"

        predicted_segmentation = []
        softmax_pred = []

        for s in range(x.shape[1]):
            pred_seg, softmax_pres = self._internal_predict_2D_2Dconv_tiled(
                x[:, s], step_size, do_mirroring, mirror_axes, patch_size, regions_class_order, use_gaussian,
                pad_border_mode, pad_kwargs, all_in_gpu, verbose)

            predicted_segmentation.append(pred_seg[None])
            softmax_pred.append(softmax_pres[None])

        predicted_segmentation = np.vstack(predicted_segmentation)
        softmax_pred = np.vstack(softmax_pred).transpose((1, 0, 2, 3))

        return predicted_segmentation, softmax_pred


if __name__ == '__main__':
    print(SegmentationNetwork._compute_steps_for_sliding_window((30, 224, 224), (162, 529, 529), 0.5))
    print(SegmentationNetwork._compute_steps_for_sliding_window((30, 224, 224), (162, 529, 529), 1))
    print(SegmentationNetwork._compute_steps_for_sliding_window((30, 224, 224), (162, 529, 529), 0.1))

    print(SegmentationNetwork._compute_steps_for_sliding_window((30, 224, 224), (60, 448, 224), 1))
    print(SegmentationNetwork._compute_steps_for_sliding_window((30, 224, 224), (60, 448, 224), 0.5))

    print(SegmentationNetwork._compute_steps_for_sliding_window((30, 224, 224), (30, 224, 224), 1))
    print(SegmentationNetwork._compute_steps_for_sliding_window((30, 224, 224), (30, 224, 224), 0.125))


    print(SegmentationNetwork._compute_steps_for_sliding_window((123, 54, 123), (246, 162, 369), 0.25))




[[0, 15, 29, 44, 59, 73, 88, 103, 117, 132], [0, 102, 203, 305], [0, 102, 203, 305]]
[[0, 26, 53, 79, 106, 132], [0, 152, 305], [0, 152, 305]]
[[0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132], [0, 22, 44, 65, 87, 109, 131, 152, 174, 196, 218, 240, 261, 283, 305], [0, 22, 44, 65, 87, 109, 131, 152, 174, 196, 218, 240, 261, 283, 305]]
[[0, 30], [0, 224], [0]]
[[0, 15, 30], [0, 112, 224], [0]]
[[0], [0], [0]]
[[0], [0], [0]]
[[0, 31, 62, 92, 123], [0, 14, 27, 40, 54, 68, 81, 94, 108], [0, 31, 62, 92, 123, 154, 184, 215, 246]]


<ipython-input-7-99e559c65698>:22: DeprecationWarning: Please import `gaussian_filter` from the `scipy.ndimage` namespace; the `scipy.ndimage.filters` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.filters import gaussian_filter


####dynunet_block

In [8]:
from typing import Optional, Sequence, Tuple, Union

import numpy as np
import torch
import torch.nn as nn

from monai.networks.blocks.convolutions import Convolution
from monai.networks.layers.factories import Act, Norm
from monai.networks.layers.utils import get_act_layer, get_norm_layer


class UnetResBlock(nn.Module):
    """
    A skip-connection based module that can be used for DynUNet, based on:
    `Automated Design of Deep Learning Methods for Biomedical Image Segmentation <https://arxiv.org/abs/1904.08128>`_.
    `nnU-Net: Self-adapting Framework for U-Net-Based Medical Image Segmentation <https://arxiv.org/abs/1809.10486>`_.

    Args:
        spatial_dims: number of spatial dimensions.
        in_channels: number of input channels.
        out_channels: number of output channels.
        kernel_size: convolution kernel size.
        stride: convolution stride.
        norm_name: feature normalization type and arguments.
        act_name: activation layer type and arguments.
        dropout: dropout probability.

    """

    def __init__(
        self,
        spatial_dims: int,
        in_channels: int,
        out_channels: int,
        kernel_size: Union[Sequence[int], int],
        stride: Union[Sequence[int], int],
        norm_name: Union[Tuple, str],
        act_name: Union[Tuple, str] = ("leakyrelu", {"inplace": True, "negative_slope": 0.01}),
        dropout: Optional[Union[Tuple, str, float]] = None,
    ):
        super().__init__()
        self.conv1 = get_conv_layer(
            spatial_dims,
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            dropout=dropout,
            conv_only=True,
        )
        self.conv2 = get_conv_layer(
            spatial_dims, out_channels, out_channels, kernel_size=kernel_size, stride=1, dropout=dropout, conv_only=True
        )
        self.lrelu = get_act_layer(name=act_name)
        self.norm1 = get_norm_layer(name=norm_name, spatial_dims=spatial_dims, channels=out_channels)
        self.norm2 = get_norm_layer(name=norm_name, spatial_dims=spatial_dims, channels=out_channels)
        self.downsample = in_channels != out_channels
        stride_np = np.atleast_1d(stride)
        if not np.all(stride_np == 1):
            self.downsample = True
        if self.downsample:
            self.conv3 = get_conv_layer(
                spatial_dims, in_channels, out_channels, kernel_size=1, stride=stride, dropout=dropout, conv_only=True
            )
            self.norm3 = get_norm_layer(name=norm_name, spatial_dims=spatial_dims, channels=out_channels)

    def forward(self, inp):
        residual = inp
        out = self.conv1(inp)
        out = self.norm1(out)
        out = self.lrelu(out)
        out = self.conv2(out)
        out = self.norm2(out)
        if hasattr(self, "conv3"):
            residual = self.conv3(residual)
        if hasattr(self, "norm3"):
            residual = self.norm3(residual)
        out += residual
        out = self.lrelu(out)
        return out


class UnetBasicBlock(nn.Module):
    """
    A CNN module module that can be used for DynUNet, based on:
    `Automated Design of Deep Learning Methods for Biomedical Image Segmentation <https://arxiv.org/abs/1904.08128>`_.
    `nnU-Net: Self-adapting Framework for U-Net-Based Medical Image Segmentation <https://arxiv.org/abs/1809.10486>`_.

    Args:
        spatial_dims: number of spatial dimensions.
        in_channels: number of input channels.
        out_channels: number of output channels.
        kernel_size: convolution kernel size.
        stride: convolution stride.
        norm_name: feature normalization type and arguments.
        act_name: activation layer type and arguments.
        dropout: dropout probability.

    """

    def __init__(
        self,
        spatial_dims: int,
        in_channels: int,
        out_channels: int,
        kernel_size: Union[Sequence[int], int],
        stride: Union[Sequence[int], int],
        norm_name: Union[Tuple, str],
        act_name: Union[Tuple, str] = ("leakyrelu", {"inplace": True, "negative_slope": 0.01}),
        dropout: Optional[Union[Tuple, str, float]] = None,
    ):
        super().__init__()
        self.conv1 = get_conv_layer(
            spatial_dims,
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            dropout=dropout,
            conv_only=True,
        )
        self.conv2 = get_conv_layer(
            spatial_dims, out_channels, out_channels, kernel_size=kernel_size, stride=1, dropout=dropout, conv_only=True
        )
        self.lrelu = get_act_layer(name=act_name)
        self.norm1 = get_norm_layer(name=norm_name, spatial_dims=spatial_dims, channels=out_channels)
        self.norm2 = get_norm_layer(name=norm_name, spatial_dims=spatial_dims, channels=out_channels)

    def forward(self, inp):
        out = self.conv1(inp)
        out = self.norm1(out)
        out = self.lrelu(out)
        out = self.conv2(out)
        out = self.norm2(out)
        out = self.lrelu(out)
        return out


class UnetUpBlock(nn.Module):
    """
    An upsampling module that can be used for DynUNet, based on:
    `Automated Design of Deep Learning Methods for Biomedical Image Segmentation <https://arxiv.org/abs/1904.08128>`_.
    `nnU-Net: Self-adapting Framework for U-Net-Based Medical Image Segmentation <https://arxiv.org/abs/1809.10486>`_.

    Args:
        spatial_dims: number of spatial dimensions.
        in_channels: number of input channels.
        out_channels: number of output channels.
        kernel_size: convolution kernel size.
        stride: convolution stride.
        upsample_kernel_size: convolution kernel size for transposed convolution layers.
        norm_name: feature normalization type and arguments.
        act_name: activation layer type and arguments.
        dropout: dropout probability.
        trans_bias: transposed convolution bias.

    """

    def __init__(
        self,
        spatial_dims: int,
        in_channels: int,
        out_channels: int,
        kernel_size: Union[Sequence[int], int],
        stride: Union[Sequence[int], int],
        upsample_kernel_size: Union[Sequence[int], int],
        norm_name: Union[Tuple, str],
        act_name: Union[Tuple, str] = ("leakyrelu", {"inplace": True, "negative_slope": 0.01}),
        dropout: Optional[Union[Tuple, str, float]] = None,
        trans_bias: bool = False,
    ):
        super().__init__()
        upsample_stride = upsample_kernel_size
        self.transp_conv = get_conv_layer(
            spatial_dims,
            in_channels,
            out_channels,
            kernel_size=upsample_kernel_size,
            stride=upsample_stride,
            dropout=dropout,
            bias=trans_bias,
            conv_only=True,
            is_transposed=True,
        )
        self.conv_block = UnetBasicBlock(
            spatial_dims,
            out_channels + out_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=1,
            dropout=dropout,
            norm_name=norm_name,
            act_name=act_name,
        )

    def forward(self, inp, skip):
        # number of channels for skip should equals to out_channels
        out = self.transp_conv(inp)
        out = torch.cat((out, skip), dim=1)
        out = self.conv_block(out)
        return out


class UnetOutBlock(nn.Module):
    def __init__(
        self, spatial_dims: int, in_channels: int, out_channels: int, dropout: Optional[Union[Tuple, str, float]] = None
    ):
        super().__init__()
        self.conv = get_conv_layer(
            spatial_dims, in_channels, out_channels, kernel_size=1, stride=1, dropout=dropout, bias=True, conv_only=True
        )

    def forward(self, inp):
        return self.conv(inp)


def get_conv_layer(
    spatial_dims: int,
    in_channels: int,
    out_channels: int,
    kernel_size: Union[Sequence[int], int] = 3,
    stride: Union[Sequence[int], int] = 1,
    act: Optional[Union[Tuple, str]] = Act.PRELU,
    norm: Union[Tuple, str] = Norm.INSTANCE,
    dropout: Optional[Union[Tuple, str, float]] = None,
    bias: bool = False,
    conv_only: bool = True,
    is_transposed: bool = False,
):
    padding = get_padding(kernel_size, stride)
    output_padding = None
    if is_transposed:
        output_padding = get_output_padding(kernel_size, stride, padding)
    return Convolution(
        spatial_dims,
        in_channels,
        out_channels,
        strides=stride,
        kernel_size=kernel_size,
        act=act,
        norm=norm,
        dropout=dropout,
        bias=bias,
        conv_only=conv_only,
        is_transposed=is_transposed,
        padding=padding,
        output_padding=output_padding,
    )


def get_padding(
    kernel_size: Union[Sequence[int], int], stride: Union[Sequence[int], int]
) -> Union[Tuple[int, ...], int]:

    kernel_size_np = np.atleast_1d(kernel_size)
    stride_np = np.atleast_1d(stride)
    padding_np = (kernel_size_np - stride_np + 1) / 2
    if np.min(padding_np) < 0:
        raise AssertionError("padding value should not be negative, please change the kernel size and/or stride.")
    padding = tuple(int(p) for p in padding_np)

    return padding if len(padding) > 1 else padding[0]


def get_output_padding(
    kernel_size: Union[Sequence[int], int], stride: Union[Sequence[int], int], padding: Union[Sequence[int], int]
) -> Union[Tuple[int, ...], int]:
    kernel_size_np = np.atleast_1d(kernel_size)
    stride_np = np.atleast_1d(stride)
    padding_np = np.atleast_1d(padding)

    out_padding_np = 2 * padding_np + stride_np - kernel_size_np
    if np.min(out_padding_np) < 0:
        raise AssertionError("out_padding value should not be negative, please change the kernel size and/or stride.")
    out_padding = tuple(int(p) for p in out_padding_np)

    return out_padding if len(out_padding) > 1 else out_padding[0]

####synapse.transformerblock

In [9]:
import torch.nn as nn
import torch
#from network_architecture.dynunet_block import UnetResBlock


class TransformerBlock(nn.Module):
    """
    A transformer block, based on: "Shaker et al.,
    UNETR++: Delving into Efficient and Accurate 3D Medical Image Segmentation"
    """

    def __init__(
            self,
            spatial_dims: int,
            input_size: int,
            hidden_size: int,
            proj_size: int,
            num_heads: int,
            dropout_rate: float = 0.0,
            pos_embed=False,
    ) -> None:
        """
        Args:
            input_size: the size of the input for each stage.
            hidden_size: dimension of hidden layer.
            proj_size: projection size for keys and values in the spatial attention module.
            num_heads: number of attention heads.
            dropout_rate: faction of the input units to drop.
            pos_embed: bool argument to determine if positional embedding is used.

        """

        super().__init__()

        if not (0 <= dropout_rate <= 1):
            raise ValueError("dropout_rate should be between 0 and 1.")

        if hidden_size % num_heads != 0:
            print("Hidden size is ", hidden_size)
            print("Num heads is ", num_heads)
            raise ValueError("hidden_size should be divisible by num_heads.")

        self.norm = nn.LayerNorm(hidden_size)
        self.gamma = nn.Parameter(1e-6 * torch.ones(hidden_size), requires_grad=True)
        self.epa_block = EPA(input_size=input_size, hidden_size=hidden_size, proj_size=proj_size, num_heads=num_heads, channel_attn_drop=dropout_rate,spatial_attn_drop=dropout_rate)
        self.conv51 = UnetResBlock(spatial_dims, hidden_size, hidden_size, kernel_size=3, stride=1, norm_name="batch")
        self.conv8 = nn.Sequential(nn.Dropout2d(0.1, False), nn.Conv2d(hidden_size, hidden_size, 1))

        self.pos_embed = None
        if pos_embed:
            self.pos_embed = nn.Parameter(torch.zeros(1, input_size, hidden_size))

    def forward(self, x):
        B, C, H, W = x.shape

        x = x.reshape(B, C, H * W).permute(0, 2, 1)

        if self.pos_embed is not None:
            x = x + self.pos_embed
        attn = x + self.gamma * self.epa_block(self.norm(x))

        attn_skip = attn.reshape(B, H, W, C).permute(0, 3, 1, 2)  # (B, C, H, W)
        attn = self.conv51(attn_skip)
        x = attn_skip + self.conv8(attn)

        return x


class EPA(nn.Module):
    """
        Efficient Paired Attention Block, based on: "Shaker et al.,
        UNETR++: Delving into Efficient and Accurate 3D Medical Image Segmentation"
        """
    def __init__(self, input_size, hidden_size, proj_size, num_heads=4, qkv_bias=False,
                 channel_attn_drop=0.1, spatial_attn_drop=0.1):
        super().__init__()
        self.num_heads = num_heads
        self.temperature = nn.Parameter(torch.ones(num_heads, 1, 1))
        self.temperature2 = nn.Parameter(torch.ones(num_heads, 1, 1))

        # qkvv are 4 linear layers (query_shared, key_shared, value_spatial, value_channel)
        self.qkvv = nn.Linear(hidden_size, hidden_size * 4, bias=qkv_bias)

        # E and F are projection matrices with shared weights used in spatial attention module to project
        # keys and values from HWD-dimension to P-dimension
        self.E = self.F = nn.Linear(input_size, proj_size)

        self.attn_drop = nn.Dropout(channel_attn_drop)
        self.attn_drop_2 = nn.Dropout(spatial_attn_drop)

        self.out_proj = nn.Linear(hidden_size, int(hidden_size // 2))
        self.out_proj2 = nn.Linear(hidden_size, int(hidden_size // 2))

    def forward(self, x):
        B, N, C = x.shape

        qkvv = self.qkvv(x)
        qkvv = qkvv.reshape(B, N, 4, self.num_heads, C // self.num_heads)

        qkvv = qkvv.permute(2, 0, 3, 1, 4)

        q_shared, k_shared, v_CA, v_SA = qkvv[0], qkvv[1], qkvv[2], qkvv[3]

        q_shared = q_shared.transpose(-2, -1)
        k_shared = k_shared.transpose(-2, -1)
        v_CA = v_CA.transpose(-2, -1)
        v_SA = v_SA.transpose(-2, -1)

        k_shared_projected = self.E(k_shared)

        v_SA_projected = self.F(v_SA)

        q_shared = torch.nn.functional.normalize(q_shared, dim=-1)
        k_shared = torch.nn.functional.normalize(k_shared, dim=-1)

        attn_CA = (q_shared @ k_shared.transpose(-2, -1)) * self.temperature

        attn_CA = attn_CA.softmax(dim=-1)
        attn_CA = self.attn_drop(attn_CA)

        x_CA = (attn_CA @ v_CA).permute(0, 3, 1, 2).reshape(B, N, C)

        attn_SA = (q_shared.permute(0, 1, 3, 2) @ k_shared_projected) * self.temperature2

        attn_SA = attn_SA.softmax(dim=-1)
        attn_SA = self.attn_drop_2(attn_SA)

        x_SA = (attn_SA @ v_SA_projected.transpose(-2, -1)).permute(0, 3, 1, 2).reshape(B, N, C)

        # Concat fusion
        x_SA = self.out_proj(x_SA)
        x_CA = self.out_proj2(x_CA)
        x = torch.cat((x_SA, x_CA), dim=-1)
        return x

    @torch.jit.ignore
    def no_weight_decay(self):
        return {'temperature', 'temperature2'}

####layers

In [10]:
import torch
import torch.nn.functional as F
from torch import nn
import math


class LayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError
        self.normalized_shape = (normalized_shape,)

    def forward(self, x):
        if self.data_format == "channels_last":
            return F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdim=True)
            s = (x - u).pow(2).mean(1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.eps)
            x = self.weight[:, None, None] * x + self.bias[:, None, None]
            return x


class PositionalEncodingFourier(nn.Module):
    def __init__(self, hidden_dim=32, dim=768, temperature=10000):
        super().__init__()
        self.token_projection = nn.Conv2d(hidden_dim * 2, dim, kernel_size=1)
        self.scale = 2 * math.pi
        self.temperature = temperature
        self.hidden_dim = hidden_dim
        self.dim = dim

    def forward(self, B, H, W):
        mask = torch.zeros(B, H, W).bool().to(self.token_projection.weight.device)
        not_mask = ~mask
        y_embed = not_mask.cumsum(1, dtype=torch.float32)
        x_embed = not_mask.cumsum(2, dtype=torch.float32)
        eps = 1e-6
        y_embed = y_embed / (y_embed[:, -1:, :] + eps) * self.scale
        x_embed = x_embed / (x_embed[:, :, -1:] + eps) * self.scale

        dim_t = torch.arange(self.hidden_dim, dtype=torch.float32, device=mask.device)
        dim_t = self.temperature ** (2 * (dim_t // 2) / self.hidden_dim)

        pos_x = x_embed[:, :, :, None] / dim_t
        pos_y = y_embed[:, :, :, None] / dim_t
        pos_x = torch.stack((pos_x[:, :, :, 0::2].sin(),
                             pos_x[:, :, :, 1::2].cos()), dim=4).flatten(3)
        pos_y = torch.stack((pos_y[:, :, :, 0::2].sin(),
                             pos_y[:, :, :, 1::2].cos()), dim=4).flatten(3)
        pos = torch.cat((pos_y, pos_x), dim=3).permute(0, 3, 1, 2)
        pos = self.token_projection(pos)

        return pos

####synapse.model_components

In [11]:
import torch
from torch import nn
from timm.models.layers import trunc_normal_
from typing import Sequence, Tuple, Union
from monai.networks.layers.utils import get_norm_layer
from monai.utils import optional_import
#from network_architecture.layers import LayerNorm
#from network_architecture.synapse.transformerblock import TransformerBlock
#from network_architecture.dynunet_block import get_conv_layer, UnetResBlock


einops, _ = optional_import("einops")
class GroupNorm(nn.Module):
    def __init__(self, num_groups, in_channels):
        super(GroupNorm, self).__init__()
        self.GN = nn.GroupNorm(num_groups=num_groups, in_channels=in_channels)
        self.GN.weight.data.fill_(1)
        self.GN.bias.data.zero_()
    def forward(self, x):
        x = self.GN(x)
        return x

class Conv3D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding=0, bias=False):
        super(Conv3D, self).__init__()
        padding = padding
        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias)

        nn.init.xavier_normal_(self.conv.weight.data)
        if bias !=False:
            nn.init.constant_(self.conv.bias.data, 0.0)
    def forward(self, x):
        x = self.conv(x)
        return x
class DConvGNLeaky(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride,  return_module=False):
        super(DConvGNLeaky, self).__init__()
        self.deconv = nn.ConvTranspose3d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, bias=True)
        self.bn1 = GroupNorm(out_channels)
        self.relu = nn.LeakyReLU(0.1, inplace=True)
        nn.init.xavier_normal_(self.deconv.weight.data)
        nn.init.constant_(self.deconv.bias.data, 0.0)

    def forward(self, x):
        x = self.deconv(x)
        x = self.bn1(x)
        x = self.relu(x)
        return x



class UnetrPPEncoder(nn.Module):
    def __init__(self, input_size, patch_size, dims=[32, 64, 128, 256],
                 proj_size =[128,128,64,32], depths=[3, 3, 3, 3],  num_heads=4, spatial_dims=2, in_channels=1, dropout=0.0, transformer_dropout_rate=0.15 ,**kwargs):
        super().__init__()

        self.downsample_layers = nn.ModuleList()  # stem and 3 intermediate downsampling conv layers
        stem_layer = nn.Sequential(
            get_conv_layer(spatial_dims, in_channels, dims[0], kernel_size=patch_size, stride=patch_size,
                           dropout=dropout, conv_only=True, ),
            get_norm_layer(name=("group", {"num_groups": 1}), channels=dims[0]),
        )
        self.downsample_layers.append(stem_layer)
        for i in range(3):
            downsample_layer = nn.Sequential(
                get_conv_layer(spatial_dims, dims[i], dims[i + 1], kernel_size=(2, 2), stride=(2, 2),
                               dropout=dropout, conv_only=True, ),
                get_norm_layer(name=("group", {"num_groups": dims[i]}), channels=dims[i + 1]),
            )
            self.downsample_layers.append(downsample_layer)

        self.stages = nn.ModuleList()  # 4 feature resolution stages, each consisting of multiple Transformer blocks
        for i in range(4):
            stage_blocks = []
            for j in range(depths[i]):
                stage_blocks.append(TransformerBlock(spatial_dims=spatial_dims, input_size=input_size[i], hidden_size=dims[i],  proj_size=proj_size[i], num_heads=num_heads,
                                     dropout_rate=transformer_dropout_rate, pos_embed=True))
            self.stages.append(nn.Sequential(*stage_blocks))
        self.hidden_states = []
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            trunc_normal_(m.weight, std=.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, (LayerNorm, nn.LayerNorm)):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward_features(self, x):
        hidden_states = []

        x = self.downsample_layers[0](x)
        x = self.stages[0](x)

        hidden_states.append(x)

        for i in range(1, 4):
            x = self.downsample_layers[i](x)
            x = self.stages[i](x)
            if i == 3:  # Reshape the output of the last stage
                x = einops.rearrange(x, "b c h w -> b (h w) c")
            hidden_states.append(x)
        return x, hidden_states

    def forward(self, x):
        x, hidden_states = self.forward_features(x)
        return x, hidden_states


class UnetrUpBlock(nn.Module):
    def     __init__(
            self,
            spatial_dims: int,
            in_channels: int,
            out_channels: int,
            kernel_size: Union[Sequence[int], int],
            upsample_kernel_size: Union[Sequence[int], int],
            norm_name: Union[Tuple, str],
            proj_size: int = 64,
            num_heads: int = 4,
            out_size: int = 0,
            depth: int = 3,
            conv_decoder: bool = False,
    ) -> None:
        """
        Args:
            spatial_dims: number of spatial dimensions.
            in_channels: number of input channels.
            out_channels: number of output channels.
            kernel_size: convolution kernel size.
            upsample_kernel_size: convolution kernel size for transposed convolution layers.
            norm_name: feature normalization type and arguments.
            proj_size: projection size for keys and values in the spatial attention module.
            num_heads: number of heads inside each EPA module.
            out_size: spatial size for each decoder.
            depth: number of blocks for the current decoder stage.
        """

        super().__init__()
        upsample_stride = upsample_kernel_size
        self.transp_conv = get_conv_layer(
            spatial_dims,
            in_channels,
            out_channels,
            kernel_size=upsample_kernel_size,
            stride=upsample_stride,
            conv_only=True,
            is_transposed=True,
        )

        # 4 feature resolution stages, each consisting of multiple residual blocks
        self.decoder_block = nn.ModuleList()
        self.conv_decoder = conv_decoder
        # If this is the last decoder, use ConvBlock(UnetResBlock) instead of EPA_Block (see suppl. material in the paper)
        if conv_decoder == True:
            self.decoder_block.append(
                UnetResBlock(spatial_dims, out_channels, out_channels, kernel_size=kernel_size, stride=1,
                             norm_name=norm_name, ))
        else:
            stage_blocks = []
            for j in range(depth):
                stage_blocks.append(TransformerBlock(spatial_dims, input_size=out_size, hidden_size= out_channels, proj_size=proj_size, num_heads=num_heads,
                                                     dropout_rate=0.15, pos_embed=True))
            self.decoder_block.append(nn.Sequential(*stage_blocks))

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            trunc_normal_(m.weight, std=.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, (nn.LayerNorm)):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward(self, inp, skip):

        out = self.transp_conv(inp)
        # if self.conv_decoder ==True:
        #     out = torch.cat([out,skip], dim=1)
        # else:
        out = out + skip
        out = self.decoder_block[0](out)

        return out

##net

####retinal_vasuclar_net

In [12]:
import torch
from torch import nn
from typing import Tuple, Union
#from network_architecture.neural_network import SegmentationNetwork
#from network_architecture.dynunet_block import UnetOutBlock, UnetResBlock
#from network_architecture.synapse.model_components import UnetrPPEncoder, UnetrUpBlock


class RetinalVasularSeg(SegmentationNetwork):
    def __init__(
            self,
            in_channels: int,
            out_channels: int,
            img_size: [640, 640],
            feature_size: int = 16,
            hidden_size: int = 256,
            num_heads: int = 4,
            pos_embed: str = "perceptron",  # TODO: Remove the argument
            norm_name: Union[Tuple, str] = "instance",
            dropout_rate: float = 0.0,
            depths=None,
            dims=None,
            conv_op=nn.Conv3d,
            do_ds=True,

    ) -> None:
        super().__init__()
        if depths is None:
            depths = [3, 3, 3, 3]
        self.do_ds = do_ds
        self.conv_op = conv_op
        self.num_classes = out_channels
        if not (0 <= dropout_rate <= 1):
            raise AssertionError("dropout_rate should be between 0 and 1.")

        if pos_embed not in ["conv", "perceptron"]:
            raise KeyError(f"Position embedding layer of type {pos_embed} is not supported.")

        self.patch_size = (3, 3)
        self.feat_size = (
            img_size[0] // self.patch_size[0] // 8,  # 8 is the downsampling happened through the four encoders stages
            img_size[1] // self.patch_size[1] // 8,  # 8 is the downsampling happened through the four encoders stages
        )
        self.hidden_size = hidden_size
        self.input_size = [self.feat_size[0]*8*self.feat_size[1]*8, self.feat_size[0]*4*self.feat_size[1]*4,
                           self.feat_size[0]*2*self.feat_size[1]*2, self.feat_size[0]*1*self.feat_size[1]*1]
        self.unetr_pp_encoder = UnetrPPEncoder(input_size=self.input_size, patch_size=self.patch_size, dims=dims, depths=depths, num_heads=num_heads, in_channels=in_channels)

        self.prarm = nn.Parameter(torch.tensor([0.2, 0.6, 0.2]),requires_grad=True).cuda().float()
        norm_name ='batch'

        self.encoder1 = UnetResBlock(
            spatial_dims=2,
            in_channels=in_channels,
            out_channels=feature_size*2,
            kernel_size=3,
            stride=1,
            norm_name=norm_name,
        )
        self.encoder11 = UnetResBlock(
            spatial_dims=2,
            in_channels=feature_size*2,
            out_channels=feature_size,
            kernel_size=3,
            stride=1,
            norm_name=norm_name,
        )
        self.decoder5 = UnetrUpBlock(
            spatial_dims=2,
            in_channels=feature_size * 16,
            out_channels=feature_size * 8,
            kernel_size=3,
            upsample_kernel_size=2,
            norm_name=norm_name,
            out_size=img_size[0]//(self.patch_size[0]*4) * img_size[1]//(self.patch_size[1]*4),
        )
        self.decoder4 = UnetrUpBlock(
            spatial_dims=2,
            in_channels=feature_size * 8,
            out_channels=feature_size * 4,
            kernel_size=3,
            upsample_kernel_size=2,
            norm_name=norm_name,
            out_size=img_size[0]//(self.patch_size[0]*2) * img_size[1]//(self.patch_size[1]*2),
        )
        self.decoder3 = UnetrUpBlock(
            spatial_dims=2,
            in_channels=feature_size * 4,
            out_channels=feature_size * 2,
            kernel_size=3,
            upsample_kernel_size=2,
            norm_name=norm_name,
            out_size=img_size[0]//self.patch_size[0] * img_size[1]//self.patch_size[1],
        )
        self.decoder2 = UnetrUpBlock(
            spatial_dims=2,
            in_channels=feature_size * 2,
            out_channels=feature_size,
            kernel_size=3,
            upsample_kernel_size=self.patch_size,
            norm_name=norm_name,
            out_size=img_size[0] * img_size[1],
            conv_decoder=True,
        )
        self.out1 = UnetOutBlock(spatial_dims=2, in_channels=feature_size, out_channels=out_channels)
        if self.do_ds:
            self.out2 = UnetOutBlock(spatial_dims=2, in_channels=feature_size * 2, out_channels=out_channels)
            self.out3 = UnetOutBlock(spatial_dims=2, in_channels=feature_size * 4, out_channels=out_channels)

    def proj_feat(self, x, hidden_size, feat_size):
        x = x.view(x.size(0), feat_size[0], feat_size[1], hidden_size)
        x = x.permute(0, 3, 1, 2).contiguous()
        return x

    def forward(self, x_in):

        x_output, hidden_states = self.unetr_pp_encoder(x_in)

        convBlock = self.encoder1(x_in)
        convBlock = self.encoder11(convBlock)


        # Four encoders
        enc1 = hidden_states[0]
        enc2 = hidden_states[1]
        enc3 = hidden_states[2]
        enc4 = hidden_states[3]

        # Four decoders
        dec4 = self.proj_feat(enc4, self.hidden_size, self.feat_size)
        dec3 = self.decoder5(dec4, enc3)
        dec2 = self.decoder4(dec3, enc2)
        dec1 = self.decoder3(dec2, enc1)

        out = self.decoder2(dec1, convBlock)
        if self.do_ds:
            logits = [self.out1(out), self.out2(dec1), self.out3(dec2)]
        else:
            logits = self.out1(out)

        return logits

####pp_lite_seg

In [13]:
# copyright (c) 2022 PaddlePaddle Authors. All Rights Reserve.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import torch
import torch.nn as nn
import torch.nn.functional as F
import math


class ConvBN(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 padding=1,
                 bias = False,
                 **kwargs):
        super().__init__()
        self._conv = nn.Conv2d(
            in_channels, out_channels, kernel_size, stride=stride, padding=kernel_size//2 if padding else 0,
            bias = bias, **kwargs)
        self._batch_norm = nn.BatchNorm2d(out_channels, momentum=0.1)

    def forward(self, x):
        x = self._conv(x)
        x = self._batch_norm(x)
        return x


class ConvBNReLU(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size=3,
                 stride = 1,
                 padding=1,
                 bias = False,
                 **kwargs):
        super().__init__()

        self._conv = nn.Conv2d(
            in_channels, out_channels, kernel_size, stride=stride, padding=kernel_size//2 if padding else 0, bias = bias,**kwargs)

        self._batch_norm = nn.BatchNorm2d(out_channels, momentum=0.1)
        self._relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self._conv(x)
        x = self._batch_norm(x)
        x = self._relu(x)
        return x


class ConvBNRelu(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size=3,
                 stride = 1,
                 padding=1,
                 bias = False,
                 **kwargs):
        super().__init__()

        self.conv = nn.Conv2d(
            in_channels, out_channels, kernel_size, stride=stride, padding=kernel_size//2 if padding else 0, bias = bias, **kwargs)

        self.bn = nn.BatchNorm2d(out_channels, momentum=0.1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x


def avg_max_reduce_channel_helper(x, use_concat=True):
    # Reduce hw by avg and max, only support single input
    assert not isinstance(x, (list, tuple))
    # print("x before mean and max:", x.shape)
    mean_value = torch.mean(x, dim=1, keepdim=True)
    max_value = torch.max(x, dim=1, keepdim=True)[0]
    # mean_value = mean_value.unsqueeze(0)
    # print("mean max:", mean_value.shape, max_value.shape)

    if use_concat:
        res = torch.at([mean_value, max_value], dim=1)
    else:
        res = [mean_value, max_value]
    return res


def avg_max_reduce_channel(x):
    # Reduce hw by avg and max
    # Return cat([avg_ch_0, max_ch_0, avg_ch_1, max_ch_1, ...])
    if not isinstance(x, (list, tuple)):
        return avg_max_reduce_channel_helper(x)
    elif len(x) == 1:
        return avg_max_reduce_channel_helper(x[0])
    else:
        res = []
        for xi in x:
            # print(xi.shape)
            res.extend(avg_max_reduce_channel_helper(xi, False))
        # print("res:\n",)
        # for it in res:
        #     print(it.shape)
        return torch.cat(res, dim=1)


class UAFM(nn.Module):
    """
    The base of Unified Attention Fusion Module.
    Args:
        x_ch (int): The channel of x tensor, which is the low level feature.
        y_ch (int): The channel of y tensor, which is the high level feature.
        out_ch (int): The channel of output tensor.
        ksize (int, optional): The kernel size of the conv for x tensor. Default: 3.
        resize_mode (str, optional): The resize model in unsampling y tensor. Default: bilinear.
    """

    def __init__(self, x_ch, y_ch, out_ch, ukernel_size=2, ksize=3, resize_mode='nearest'):
        super().__init__()

        self.conv_x = ConvBNReLU(
            x_ch, y_ch, kernel_size=ksize, padding=ksize // 2, bias=False)
        self.conv_out = ConvBNReLU(
            y_ch, out_ch, kernel_size=3, padding=1, bias=False)
        self.resize_mode = resize_mode
        self.transp = nn.ConvTranspose2d(y_ch, y_ch, kernel_size=ukernel_size, stride=ukernel_size, bias=False)

    def check(self, x, y):
        # print("x dim:",x.ndim)
        assert x.ndim == 4 and y.ndim == 4
        x_h, x_w = x.shape[2:]
        y_h, y_w = y.shape[2:]
        assert x_h >= y_h and x_w >= y_w

    def prepare(self, x, y):
        x = self.prepare_x(x, y)
        y = self.prepare_y(x, y)
        return x, y

    def prepare_x(self, x, y):
        x = self.conv_x(x)
        return x

    def prepare_y(self, x, y):
        y_up = self.transp(y)#F.interpolate(y, x.shape[2:], mode=self.resize_mode)
        return y_up

    def fuse(self, x, y):
        out = x + y
        out = self.conv_out(out)
        return out

    def forward(self, x, y):
        """
        Args:
            x (Tensor): The low level feature.
            y (Tensor): The high level feature.
        """
        # print("x,y shape:",x.shape, y.shape)
        self.check(x, y)
        x, y = self.prepare(x, y)
        out = self.fuse(x, y)
        return out


class UAFM_SpAtten(UAFM):
    """
    The UAFM with spatial attention, which uses mean and max values.
    Args:
        x_ch (int): The channel of x tensor, which is the low level feature.
        y_ch (int): The channel of y tensor, which is the high level feature.
        out_ch (int): The channel of output tensor.
        ksize (int, optional): The kernel size of the conv for x tensor. Default: 3.
        resize_mode (str, optional): The resize model in unsampling y tensor. Default: bilinear.
    """

    def __init__(self, x_ch, y_ch, out_ch, ukernel_size=2, ksize=3, resize_mode='bilinear'):
        super().__init__(x_ch, y_ch, out_ch, ukernel_size, ksize, resize_mode)

        self.conv_xy_atten = nn.Sequential(
            ConvBNReLU(
                4, 2, kernel_size=3, padding=1, bias=False),
            ConvBN(
                2, 1, kernel_size=3, padding=1, bias=False))

    def fuse(self, x, y):
        """
        Args:
            x (Tensor): The low level feature.
            y (Tensor): The high level feature.
        """
        # print("x, y shape:",x.shape, y.shape)
        atten = avg_max_reduce_channel([x, y])
        atten = F.sigmoid(self.conv_xy_atten(atten))

        out = x * atten + y * (1 - atten)
        # out = self.conv_out(out)
        return out


class CatBottleneck(nn.Module):
    def __init__(self, in_planes, out_planes, block_num=3, stride=1):
        super(CatBottleneck, self).__init__()
        assert block_num > 1, "block number should be larger than 1."
        self.conv_list = nn.ModuleList()
        self.stride = stride
        if stride == 2:
            self.avd_layer = nn.Sequential(
                nn.Conv2d(
                    out_planes // 2,
                    out_planes // 2,
                    kernel_size=3,
                    stride=2,
                    padding=1,
                    groups=out_planes // 2,
                    bias=False),
                nn.BatchNorm2d(out_planes // 2, momentum=0.1), )
            self.skip = nn.AvgPool2d(kernel_size=3, stride=2, padding=1)
            stride = 1

        for idx in range(block_num):
            if idx == 0:
                self.conv_list.append(
                    ConvBNRelu(
                        in_planes, out_planes // 2, kernel_size=1))
            elif idx == 1 and block_num == 2:
                self.conv_list.append(
                    ConvBNRelu(
                        out_planes // 2, out_planes // 2, stride=stride))
            elif idx == 1 and block_num > 2:
                self.conv_list.append(
                    ConvBNRelu(
                        out_planes // 2, out_planes // 4, stride=stride))
            elif idx < block_num - 1:
                self.conv_list.append(
                    ConvBNRelu(out_planes // int(math.pow(2, idx)), out_planes
                               // int(math.pow(2, idx + 1))))
            else:
                self.conv_list.append(
                    ConvBNRelu(out_planes // int(math.pow(2, idx)), out_planes
                               // int(math.pow(2, idx))))

    def forward(self, x):
        out_list = []
        out1 = self.conv_list[0](x)
        for idx, conv in enumerate(self.conv_list[1:]):
            if idx == 0:
                if self.stride == 2:
                    out = conv(self.avd_layer(out1))
                else:
                    out = conv(out1)
            else:
                out = conv(out)
            out_list.append(out)

        if self.stride == 2:
            out1 = self.skip(out1)
        out_list.insert(0, out1)
        out = torch.cat(out_list, dim=1)
        return out


class AddBottleneck(nn.Module):
    def __init__(self, in_planes, out_planes, block_num=3, stride=1):
        super(AddBottleneck, self).__init__()
        assert block_num > 1, "block number should be larger than 1."
        self.conv_list = nn.ModuleList()
        self.stride = stride
        if stride == 2:
            self.avd_layer = nn.Sequential(
                nn.Conv2d(
                    out_planes // 2,
                    out_planes // 2,
                    kernel_size=3,
                    stride=2,
                    padding=1,
                    groups=out_planes // 2,
                    bias=False),
                nn.BatchNorm2D(out_planes // 2, momentum=0.1), )
            self.skip = nn.Sequential(
                nn.Conv2d(
                    in_planes,
                    in_planes,
                    kernel_size=3,
                    stride=2,
                    padding=1,
                    groups=in_planes,
                    bias_attr=False),
                nn.BatchNorm2d(in_planes, momentum=0.1),
                nn.Conv2d(
                    in_planes, out_planes, kernel_size=1, bias=False),
                nn.BatchNorm2d(out_planes, momentum=0.1), )
            stride = 1

        for idx in range(block_num):
            if idx == 0:
                self.conv_list.append(
                    ConvBNRelu(
                        in_planes, out_planes // 2, kernel=1))
            elif idx == 1 and block_num == 2:
                self.conv_list.append(
                    ConvBNRelu(
                        out_planes // 2, out_planes // 2, stride=stride))
            elif idx == 1 and block_num > 2:
                self.conv_list.append(
                    ConvBNRelu(
                        out_planes // 2, out_planes // 4, stride=stride))
            elif idx < block_num - 1:
                self.conv_list.append(
                    ConvBNRelu(out_planes // int(math.pow(2, idx)), out_planes
                               // int(math.pow(2, idx + 1))))
            else:
                self.conv_list.append(
                    ConvBNRelu(out_planes // int(math.pow(2, idx)), out_planes
                               // int(math.pow(2, idx))))

    def forward(self, x):
        out_list = []
        out = x
        for idx, conv in enumerate(self.conv_list):
            if idx == 0 and self.stride == 2:
                out = self.avd_layer(conv(out))
            else:
                out = conv(out)
            out_list.append(out)
        if self.stride == 2:
            x = self.skip(x)
        return torch.cat(out_list, dim=1) + x


class STDCNet(nn.Module):
    """
    The STDCNet implementation based on Pytorch.

    The original article refers to Meituan
    Fan, Mingyuan, et al. "Rethinking BiSeNet For Real-time Semantic Segmentation."
    (https://arxiv.org/abs/2104.13188)

    Args:
        base(int, optional): base channels. Default: 64.
        layers(list, optional): layers numbers list. It determines STDC block numbers of STDCNet's stage3\4\5. Defualt: [4, 5, 3].
        block_num(int,optional): block_num of features block. Default: 4.
        type(str,optional): feature fusion method "cat"/"add". Default: "cat".
        num_classes(int, optional): class number for image classification. Default: 1000.
        dropout(float,optional): dropout ratio. if >0,use dropout ratio.  Default: 0.20.
        use_conv_last(bool,optional): whether to use the last ConvBNReLU layer . Default: False.
        pretrained(str, optional): the path of pretrained model.
    """

    def __init__(self,
                 base=64,
                 layers=[4, 5, 3],
                 block_num=4,
                 type="cat",
                 num_classes=1000,
                 dropout=0.20,
                 use_conv_last=False,
                 pretrained=None):
        super(STDCNet, self).__init__()
        if type == "cat":
            block = CatBottleneck
        elif type == "add":
            block = AddBottleneck
        self.use_conv_last = use_conv_last
        self.feat_channels = [base // 2, base, base * 4, base * 8, base * 16]
        self.features = self._make_layers(base, layers, block_num, block)
        self.conv_last = ConvBNRelu(base * 16, max(1024, base * 16), 1, 1)

        if (layers == [4, 5, 3]):  # stdc1446
            self.x2 = nn.Sequential(self.features[:1])
            self.x4 = nn.Sequential(self.features[1:2])
            self.x8 = nn.Sequential(self.features[2:6])
            self.x16 = nn.Sequential(self.features[6:11])
            self.x32 = nn.Sequential(self.features[11:])
        elif (layers == [2, 2, 2]):  # stdc813
            self.x2 = nn.Sequential(self.features[:1])
            self.x4 = nn.Sequential(self.features[1:2])
            self.x8 = nn.Sequential(self.features[2:4])
            self.x16 = nn.Sequential(self.features[4:6])
            self.x32 = nn.Sequential(self.features[6:])
        else:
            raise NotImplementedError(
                "model with layers:{} is not implemented!".format(layers))

        self.pretrained = pretrained
        # self.init_weight()

    def forward(self, x):
        """
        forward function for feature extract.
        """
        feat2 = self.x2(x)
        feat4 = self.x4(feat2)
        feat8 = self.x8(feat4)
        feat16 = self.x16(feat8)
        feat32 = self.x32(feat16)
        if self.use_conv_last:
            feat32 = self.conv_last(feat32)
        return feat2, feat4, feat8, feat16, feat32

    def _make_layers(self, base, layers, block_num, block):
        features = []
        features += [ConvBNRelu(3, base // 2, 3, 2)]
        features += [ConvBNRelu(base // 2, base, 3, 2)]

        for i, layer in enumerate(layers):
            for j in range(layer):
                if i == 0 and j == 0:
                    features.append(block(base, base * 4, block_num, 2))
                elif j == 0:
                    features.append(
                        block(base * int(math.pow(2, i + 1)), base * int(
                            math.pow(2, i + 2)), block_num, 2))
                else:
                    features.append(
                        block(base * int(math.pow(2, i + 2)), base * int(
                            math.pow(2, i + 2)), block_num, 1))

        return nn.Sequential(*features)

    # def init_weight(self):
    #     for layer in self.sublayers():
    #         if isinstance(layer, nn.Conv2D):
    #             param_init.normal_init(layer.weight, std=0.001)
    #         elif isinstance(layer, (nn.BatchNorm, nn.SyncBatchNorm)):
    #             param_init.constant_init(layer.weight, value=1.0)
    #             param_init.constant_init(layer.bias, value=0.0)
    #     if self.pretrained is not None:
    #         utils.load_pretrained_model(self, self.pretrained)


def STDC2(**kwargs):
    model = STDCNet(base=64, layers=[4, 5, 3], **kwargs)
    return model


class PPLiteSeg(nn.Module):
    """
    The PP_LiteSeg implementation based on Pytorch.

    The original article refers to "Juncai Peng, Yi Liu, Shiyu Tang, Yuying Hao, Lutao Chu,
    Guowei Chen, Zewu Wu, Zeyu Chen, Zhiliang Yu, Yuning Du, Qingqing Dang,Baohua Lai,
    Qiwen Liu, Xiaoguang Hu, Dianhai Yu, Yanjun Ma. PP-LiteSeg: A Superior Real-Time Semantic
    Segmentation Model. https://arxiv.org/abs/2204.02681".

    Args:
        num_classes (int): The number of target classes.
        backbone(nn.Layer): Backbone network, such as stdc1net and resnet18. The backbone must
            has feat_channels, of which the length is 5.
        backbone_indices (List(int), optional): The values indicate the indices of output of backbone.
            Default: [2, 3, 4].
        arm_type (str, optional): The type of attention refinement module. Default: ARM_Add_SpAttenAdd3.
        cm_bin_sizes (List(int), optional): The bin size of context module. Default: [1,2,4].
        cm_out_ch (int, optional): The output channel of the last context module. Default: 128.
        arm_out_chs (List(int), optional): The out channels of each arm module. Default: [64, 96, 128].
        seg_head_inter_chs (List(int), optional): The intermediate channels of segmentation head.
            Default: [64, 64, 64].
        resize_mode (str, optional): The resize mode for the upsampling operation in decoder.
            Default: bilinear.
        pretrained (str, optional): The path or url of pretrained model. Default: None.

    """

    def __init__(self,
                 num_classes = 19,
                 backbone = STDC2(),
                 backbone_indices=[2, 3, 4],
                 arm_type='UAFM_SpAtten',
                 cm_bin_sizes=[1, 2, 4],
                 cm_out_ch=128,
                 arm_out_chs=[64, 96, 128],
                 seg_head_inter_chs=[64, 64, 64],
                 resize_mode='nearest',
                 pretrained=False):
        super().__init__()

        # backbone
        assert hasattr(backbone, 'feat_channels'), \
            "The backbone should has feat_channels."
        assert len(backbone.feat_channels) >= len(backbone_indices), \
            f"The length of input backbone_indices ({len(backbone_indices)}) should not be" \
            f"greater than the length of feat_channels ({len(backbone.feat_channels)})."
        assert len(backbone.feat_channels) > max(backbone_indices), \
            f"The max value ({max(backbone_indices)}) of backbone_indices should be " \
            f"less than the length of feat_channels ({len(backbone.feat_channels)})."
        self.backbone = backbone

        assert len(backbone_indices) > 1, "The lenght of backbone_indices " \
                                          "should be greater than 1"
        self.backbone_indices = backbone_indices  # [..., x16_id, x32_id]
        backbone_out_chs = [backbone.feat_channels[i] for i in backbone_indices]

        # head
        if len(arm_out_chs) == 1:
            arm_out_chs = arm_out_chs * len(backbone_indices)
        assert len(arm_out_chs) == len(backbone_indices), "The length of " \
                                                          "arm_out_chs and backbone_indices should be equal"

        self.ppseg_head = PPLiteSegHead(backbone_out_chs, arm_out_chs,
                                        cm_bin_sizes, cm_out_ch, arm_type,
                                        resize_mode)

        if len(seg_head_inter_chs) == 1:
            seg_head_inter_chs = seg_head_inter_chs * len(backbone_indices)
        assert len(seg_head_inter_chs) == len(backbone_indices), "The length of " \
                                                                 "seg_head_inter_chs and backbone_indices should be equal"
        self.seg_heads = nn.ModuleList()  # [..., head_16, head32]
        print("arm_out_chs:",arm_out_chs, " ; seg_head_inter_chs:",seg_head_inter_chs)
        for in_ch, mid_ch in zip(arm_out_chs, seg_head_inter_chs):
            self.seg_heads.append(SegHead(in_ch, mid_ch, num_classes))

        # pretrained
        self.pretrained = pretrained
        # self.init_weight()

    def forward(self, x):
        x_hw = x.shape[2:]
        # print("x_hw:",x_hw)

        feats_backbone = self.backbone(x)  # [x2, x4, x8, x16, x32]
        # print(type(feats_backbone))
        assert len(feats_backbone) >= len(self.backbone_indices), \
            f"The nums of backbone feats ({len(feats_backbone)}) should be greater or " \
            f"equal than the nums of backbone_indices ({len(self.backbone_indices)})"

        feats_selected = [feats_backbone[i] for i in self.backbone_indices]

        feats_head = self.ppseg_head(feats_selected)  # [..., x8, x16, x32]

        if self.training:
            logit_list = []

            for x, seg_head in zip(feats_head, self.seg_heads):
                x = seg_head(x)
                logit_list.append(x)

            logit_list = [
                F.interpolate(
                    x, x_hw, mode='bilinear', align_corners=None)
                for x in logit_list
            ]
        else:
            x = self.seg_heads[0](feats_head[0])
            # print("x:",x.shape)
            x = F.interpolate(x, x_hw, mode='bilinear', align_corners=None)
            logit_list = [x]

        return logit_list

    # def init_weight(self):
    #     if self.pretrained is not None:
    #         utils.load_entire_model(self, self.pretrained)


class PPLiteSegHead(nn.Module):
    """
    The head of PPLiteSeg.

    Args:
        backbone_out_chs (List(Tensor)): The channels of output tensors in the backbone.
        arm_out_chs (List(int)): The out channels of each arm module.
        cm_bin_sizes (List(int)): The bin size of context module.
        cm_out_ch (int): The output channel of the last context module.
        arm_type (str): The type of attention refinement module.
        resize_mode (str): The resize mode for the upsampling operation in decoder.
    """

    def __init__(self, backbone_out_chs, arm_out_chs, cm_bin_sizes, cm_out_ch,
                 arm_type, resize_mode):
        super().__init__()

        self.cm = PPContextModule(backbone_out_chs[-1], cm_out_ch, cm_out_ch,
                                  cm_bin_sizes)

        # assert hasattr(layers, arm_type), \
        #     "Not support arm_type ({})".format(arm_type)
        arm_class = eval(arm_type)

        self.arm_list = nn.ModuleList()  # [..., arm8, arm16, arm32]
        for i in range(len(backbone_out_chs)):
            low_chs = backbone_out_chs[i]
            high_ch = cm_out_ch if i == len(
                backbone_out_chs) - 1 else arm_out_chs[i + 1]
            out_ch = arm_out_chs[i]
            arm = arm_class(
                low_chs, high_ch, out_ch, ksize=3, resize_mode=resize_mode)
            self.arm_list.append(arm)

    def forward(self, in_feat_list):
        """
        Args:
            in_feat_list (List(Tensor)): Such as [x2, x4, x8, x16, x32].
                x2, x4 and x8 are optional.
        Returns:
            out_feat_list (List(Tensor)): Such as [x2, x4, x8, x16, x32].
                x2, x4 and x8 are optional.
                The length of in_feat_list and out_feat_list are the same.
        """

        high_feat = self.cm(in_feat_list[-1])
        out_feat_list = []

        for i in reversed(range(len(in_feat_list))):
            low_feat = in_feat_list[i]
            arm = self.arm_list[i]
            high_feat = arm(low_feat, high_feat)
            out_feat_list.insert(0, high_feat)

        return out_feat_list


class PPContextModule(nn.Module):
    """
    Simple Context module.

    Args:
        in_channels (int): The number of input channels to pyramid pooling module.
        inter_channels (int): The number of inter channels to pyramid pooling module.
        out_channels (int): The number of output channels after pyramid pooling module.
        bin_sizes (tuple, optional): The out size of pooled feature maps. Default: (1, 3).
        align_corners (bool): An argument of F.interpolate. It should be set to False
            when the output size of feature is even, e.g. 1024x512, otherwise it is True, e.g. 769x769.
    """

    def __init__(self,
                 in_channels,
                 inter_channels,
                 out_channels,
                 bin_sizes,
                 align_corners=None):
        super().__init__()

        self.stages = nn.ModuleList([
            self._make_stage(in_channels, inter_channels, size)
            for size in bin_sizes
        ])

        self.conv_out = ConvBNReLU(
            in_channels=inter_channels,
            out_channels=out_channels,
            kernel_size=3,
            padding=1,
            bias=True)

        self.align_corners = align_corners

    def _make_stage(self, in_channels, out_channels, size):
        prior = nn.AdaptiveAvgPool2d(output_size=size)
        conv = ConvBNReLU(
            in_channels=in_channels, out_channels=out_channels, kernel_size=1, bias=True)
        return nn.Sequential(prior, conv)

    def forward(self, input):
        out = None
        input_shape = input.shape[2:]

        for stage in self.stages:
            x = stage(input)
            x = F.interpolate(
                x,
                input_shape,
                mode='nearest',
                align_corners=self.align_corners)
            if out is None:
                out = x
            else:
                out += x

        out = self.conv_out(out)
        return out


class SegHead(nn.Module):
    def __init__(self, in_chan, mid_chan, n_classes):
        super().__init__()
        self.conv = ConvBNReLU(
            in_chan,
            mid_chan,
            kernel_size=3,
            stride=1,
            padding=1,
            bias=False)
        # print("="*100)
        # print("out:",mid_chan, "n_classes:",n_classes)
        self.conv_out = nn.Conv2d(
            mid_chan, n_classes, kernel_size=1, bias=False)

    def forward(self, x):
        x = self.conv(x)
        x = self.conv_out(x)
        return x

#
# def get_seg_model(**kwargs):
#     model = PPLiteSeg(pretrained=False)
#     return model

####pos_embdb

In [14]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
# --------------------------------------------------------
# Position embedding utils
# --------------------------------------------------------

import numpy as np

import torch

# --------------------------------------------------------
# 2D sine-cosine position embedding
# References:
# Transformer: https://github.com/tensorflow/models/blob/master/official/nlp/transformer/model_utils.py
# MoCo v3: https://github.com/facebookresearch/moco-v3
# --------------------------------------------------------
def get_2d_sincos_pos_embed(embed_dim, grid_size, cls_token=False):
    """
    grid_size: int of the grid height and width
    return:
    pos_embed: [grid_size*grid_size, embed_dim] or [1+grid_size*grid_size, embed_dim] (w/ or w/o cls_token)
    """
    grid_h = np.arange(grid_size, dtype=np.float32)
    grid_w = np.arange(grid_size, dtype=np.float32)
    grid = np.meshgrid(grid_w, grid_h)  # here w goes first
    grid = np.stack(grid, axis=0)

    grid = grid.reshape([2, 1, grid_size, grid_size])
    pos_embed = get_2d_sincos_pos_embed_from_grid(embed_dim, grid)
    if cls_token:
        pos_embed = np.concatenate([np.zeros([1, embed_dim]), pos_embed], axis=0)
    return pos_embed


def get_2d_sincos_pos_embed_from_grid(embed_dim, grid):
    assert embed_dim % 2 == 0

    # use half of dimensions to encode grid_h
    emb_h = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[0])  # (H*W, D/2)
    emb_w = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[1])  # (H*W, D/2)

    emb = np.concatenate([emb_h, emb_w], axis=1) # (H*W, D)
    return emb


def get_1d_sincos_pos_embed_from_grid(embed_dim, pos):
    """
    embed_dim: output dimension for each position
    pos: a list of positions to be encoded: size (M,)
    out: (M, D)
    """
    assert embed_dim % 2 == 0
    omega = np.arange(embed_dim // 2, dtype=float)
    omega /= embed_dim / 2.
    omega = 1. / 10000**omega  # (D/2,)

    pos = pos.reshape(-1)  # (M,)
    out = np.einsum('m,d->md', pos, omega)  # (M, D/2), outer product

    emb_sin = np.sin(out) # (M, D/2)
    emb_cos = np.cos(out) # (M, D/2)

    emb = np.concatenate([emb_sin, emb_cos], axis=1)  # (M, D)
    return emb


# --------------------------------------------------------
# Interpolate position embeddings for high-resolution
# References:
# DeiT: https://github.com/facebookresearch/deit
# --------------------------------------------------------
def interpolate_pos_embed(model, checkpoint_model):
    if 'pos_embed' in checkpoint_model:
        pos_embed_checkpoint = checkpoint_model['pos_embed']
        embedding_size = pos_embed_checkpoint.shape[-1]
        num_patches = model.patch_embed.num_patches
        num_extra_tokens = model.pos_embed.shape[-2] - num_patches
        # height (== width) for the checkpoint position embedding
        orig_size = int((pos_embed_checkpoint.shape[-2] - num_extra_tokens) ** 0.5)
        # height (== width) for the new position embedding
        new_size = int(num_patches ** 0.5)
        # class_token and dist_token are kept unchanged
        if orig_size != new_size:
            print("Position interpolate from %dx%d to %dx%d" % (orig_size, orig_size, new_size, new_size))
            extra_tokens = pos_embed_checkpoint[:, :num_extra_tokens]
            # only the position tokens are interpolated
            pos_tokens = pos_embed_checkpoint[:, num_extra_tokens:]
            pos_tokens = pos_tokens.reshape(-1, orig_size, orig_size, embedding_size).permute(0, 3, 1, 2)
            pos_tokens = torch.nn.functional.interpolate(
                pos_tokens, size=(new_size, new_size), mode='bicubic', align_corners=False)
            pos_tokens = pos_tokens.permute(0, 2, 3, 1).flatten(1, 2)
            new_pos_embed = torch.cat((extra_tokens, pos_tokens), dim=1)
            checkpoint_model['pos_embed'] = new_pos_embed

####UNetFamily

In [15]:
"""
This part contains UNet series models,
including UNet, R2UNet, Attention UNet, R2Attention UNet, DenseUNet
"""
from typing import Union, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from timm.models.vision_transformer import Block
#from net.pp_lite_seg import  UAFM_SpAtten


# ==========================Core Module================================
class conv_block(nn.Module):
    def __init__(self, ch_in, ch_out):
        super(conv_block, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True),

            nn.Conv2d(ch_out, ch_out, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.conv(x)
        return x

class up_conv(nn.Module):
    def __init__(self, ch_in, ch_out, scale_factor=2):
        super(up_conv, self).__init__()
        self.up = nn.Sequential(
            nn.Upsample(scale_factor=scale_factor),
            nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.up(x)
        return x


class Recurrent_block(nn.Module):
    def __init__(self, ch_out, t=2):
        super(Recurrent_block, self).__init__()
        self.t = t
        self.ch_out = ch_out
        self.conv = nn.Sequential(
            nn.Conv2d(ch_out, ch_out, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        for i in range(self.t):

            if i == 0:
                x1 = self.conv(x)

            x1 = self.conv(x + x1)
        return x1


class RRCNN_block(nn.Module):
    def __init__(self, ch_in, ch_out, t=2):
        super(RRCNN_block, self).__init__()
        self.RCNN = nn.Sequential(
            Recurrent_block(ch_out, t=t),
            Recurrent_block(ch_out, t=t)
        )
        self.Conv_1x1 = nn.Conv2d(ch_in, ch_out, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        x = self.Conv_1x1(x)
        x1 = self.RCNN(x)
        return x + x1


class single_conv(nn.Module):
    def __init__(self, ch_in, ch_out):
        super(single_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class Attention_block(nn.Module):  # attention Gate
    def __init__(self, F_g, F_l, F_int):
        super(Attention_block, self).__init__()
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )

        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )

        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)

        return x * psi

# =====R2U增=====
def autopad(k, p=None):
    if p is None:
        p = k // 2 if isinstance(k, int) else [x // 2 for x in k]
    return p


class ConvBlock(nn.Module):

    def __init__(self,
                 input_channels: int,
                 output_channels: int,
                 kernel_size: Tuple[int, int],
                 stride: Tuple[int, int],
                 padding: Union[int, None] = None,
                 activation: bool = True,
                 **kwargs):
        super(ConvBlock, self).__init__()
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.activation = activation
        self.conv = nn.Conv2d(in_channels=self.input_channels,
                              out_channels=self.output_channels,
                              kernel_size=kernel_size,
                              stride=stride,
                              padding=autopad(k=kernel_size, p=padding), **kwargs)
        self.bn = nn.BatchNorm2d(num_features=output_channels)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = self.conv(x)
        out = self.bn(out)
        if self.activation:
            out = F.relu(out)
        return out


class ResPath(nn.Module):

    def __init__(self,
                 input_channels : int,
                 output_channels: int,
                 length: int,
                 padding: Union[int, None] = None):
        super(ResPath, self).__init__()
        self.length = length
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.conv1 = ConvBlock(input_channels=self.input_channels,
                               output_channels=self.output_channels,
                               kernel_size=(1, 1),
                               stride=(1, 1),
                               padding=autopad(k=(1, 1), p=padding),
                               activation=False)

        self.conv2 = ConvBlock(input_channels=self.input_channels,
                               output_channels=self.output_channels,
                               kernel_size=(3, 3),
                               stride=(1, 1),
                               padding=autopad(k=(3, 3), p=padding))

        self.bn = nn.BatchNorm2d(num_features=self.conv2.output_channels)

        self.module = nn.ModuleList()

        for i in range(self.length-1):
            self.module.append(module=ConvBlock(input_channels=self.output_channels,
                                                output_channels=self.output_channels,
                                                kernel_size=(1, 1),
                                                stride=(1, 1),
                                                activation=False,
                                                padding=autopad(k=(1, 1), p=padding)))
            self.module.append(module=ConvBlock(input_channels=self.output_channels,
                                                output_channels=self.output_channels,
                                                kernel_size=(3, 3),
                                                stride=(1, 1),
                                                padding=autopad(k=(3, 3), p=padding)))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        shortcut = x
        shortcut = self.conv1(shortcut)
        out = self.conv2(x)
        out = torch.add(shortcut, out)
        out = F.relu(out)
        out = self.bn(out)

        for i in range(self.length-1):
            shortcut = out
            shortcut = self.module[i*2](shortcut)
            out = self.module[i*2+1](out)
            out = torch.add(shortcut, out)
            out = F.relu(out)
            out = self.bn(out)

        return out

# ==================================================================
class U_Net(nn.Module):
    def __init__(self, img_ch=3, output_ch=2, fea_channels=32):
        super(U_Net, self).__init__()

        self.Maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.Conv1 = conv_block(ch_in=img_ch, ch_out=fea_channels)
        self.Conv2 = conv_block(ch_in=fea_channels, ch_out=fea_channels*2)
        self.Conv3 = conv_block(ch_in=fea_channels*2, ch_out=fea_channels*4)
        self.Conv4 = conv_block(ch_in=fea_channels*4, ch_out=fea_channels*8)
        self.Conv5 = conv_block(ch_in=fea_channels*8, ch_out=fea_channels*16)

        self.Up5 = up_conv(ch_in=fea_channels*16, ch_out=fea_channels*8)
        self.Up_conv5 = conv_block(ch_in=fea_channels*16, ch_out=fea_channels*8)

        self.Up4 = up_conv(ch_in=fea_channels*8, ch_out=fea_channels*4)
        self.Up_conv4 = conv_block(ch_in=fea_channels*8, ch_out=fea_channels*4)

        self.Up3 = up_conv(ch_in=fea_channels*4, ch_out=fea_channels*2)
        self.Up_conv3 = conv_block(ch_in=fea_channels*4, ch_out=fea_channels*2)

        self.Up2 = up_conv(ch_in=fea_channels*2, ch_out=fea_channels*1)
        self.Up_conv2 = conv_block(ch_in=fea_channels*2, ch_out=fea_channels*1)

        self.Conv_1x1 = nn.Conv2d(fea_channels*1, output_ch, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        # encoding path
        x1 = self.Conv1(x)
        # print('x1',x1.shape)
        x2 = self.Maxpool(x1)
        x2 = self.Conv2(x2)
        # print('x2',x2.shape)
        x3 = self.Maxpool(x2)
        x3 = self.Conv3(x3)

        x4 = self.Maxpool(x3)
        x4 = self.Conv4(x4)
        # print(x4.shape)

        x5 = self.Maxpool(x4)
        x5 = self.Conv5(x5)

        # decoding + concat path
        d5 = self.Up5(x5)
        d5 = torch.cat((x4, d5), dim=1)

        d5 = self.Up_conv5(d5)

        d4 = self.Up4(d5)
        d4 = torch.cat((x3, d4), dim=1)
        d4 = self.Up_conv4(d4)
        # print(d4.shape)

        d3 = self.Up3(d4)
        d3 = torch.cat((x2, d3), dim=1)
        d3 = self.Up_conv3(d3)

        d2 = self.Up2(d3)
        d2 = torch.cat((x1, d2), dim=1)
        d2 = self.Up_conv2(d2)

        d1 = self.Conv_1x1(d2)
        d1 = F.softmax(d1,dim=1)  # mine

        return [d1, d1,d1]

from functools import partial
#from net.pos_embdb import get_2d_sincos_pos_embed
##########################################################################################
class U_Nett(nn.Module):
    def __init__(self, img_ch=3, output_ch=2, fea_channels=64):
        super(U_Nett, self).__init__()

        self.Maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.Conv1 = conv_block(ch_in=img_ch, ch_out=fea_channels)
        self.Conv2 = conv_block(ch_in=fea_channels, ch_out=fea_channels*2)
        self.Conv3 = conv_block(ch_in=fea_channels*2, ch_out=fea_channels*4)
        self.Conv4 = conv_block(ch_in=fea_channels*4, ch_out=fea_channels*8)

        self.sizem = 24
        self.param = nn.Parameter(torch.zeros(1, self.sizem * self.sizem, 1),requires_grad=True)  # fixed sin-cos embedding
        self.pos_embed = nn.Parameter(torch.zeros(1, self.sizem*self.sizem, fea_channels*8), requires_grad=False)  # fixed sin-cos embedding
        norm_layer = partial(nn.LayerNorm, eps=1e-6)
        self.transblock = nn.ModuleList([
            Block(dim=fea_channels*8, num_heads=fea_channels*8//64, mlp_ratio=2, qkv_bias=True, norm_layer=norm_layer)
            for i in range(4)])

        # self.UaSp1 = UAFM_SpAtten(x_ch=fea_channels*4, y_ch=fea_channels*8, out_ch=fea_channels*4, ukernel_size=2, ksize=3)

        self.Up4 = up_conv(ch_in=fea_channels*8, ch_out=fea_channels*4)
        self.Up_conv4 = conv_block(ch_in=fea_channels*8, ch_out=fea_channels*4)

        # self.UaSp2 = UAFM_SpAtten(x_ch=fea_channels * 2, y_ch=fea_channels * 4, out_ch=fea_channels * 4, ukernel_size=2, ksize=3)
        self.Up3 = up_conv(ch_in=fea_channels*4, ch_out=fea_channels*2)
        self.Up_conv3 = conv_block(ch_in=fea_channels*4, ch_out=fea_channels*2)

        # self.UaSp3 = UAFM_SpAtten(x_ch=fea_channels * 1, y_ch=fea_channels * 2, out_ch=fea_channels * 2, ukernel_size=3,ksize=3)

        self.Up2 = up_conv(ch_in=fea_channels*2, ch_out=fea_channels*1, scale_factor=2)
        self.Up_conv2 = conv_block(ch_in=fea_channels*2, ch_out=fea_channels*1)

        self.Conv_1x1 = nn.Conv2d(fea_channels*1, output_ch, kernel_size=1, stride=1, padding=0)
        self.initialize_weights()
    def initialize_weights(self):
        # initialization
        # initialize (and freeze) pos_embed by sin-cos embedding
        pos_embed = get_2d_sincos_pos_embed(self.pos_embed.shape[-1], int(self.sizem), cls_token=False)
        self.pos_embed.data.copy_(torch.from_numpy(pos_embed).float().unsqueeze(0))
    def forward(self, x):
        # encoding path
        x1 = self.Conv1(x)
        # print('x1',x1.shape)
        x2 = self.Maxpool1(x1)
        x2 = self.Conv2(x2)
        # print('x2',x2.shape)
        x3 = self.Maxpool2(x2)
        x3 = self.Conv3(x3)

        x4 = self.Maxpool3(x3)
        x4 = self.Conv4(x4)

        b, c, h, w = x4.shape
        x4 = x4.view(b, c, h* w).permute(0, 2, 1)
        xc = x4+ self.pos_embed
        for cpb in self.transblock:
            xc = cpb(xc)
        x4 = (self.param*xc + x4).permute(0, 2, 1).view(b, c, h, w)


        #d4 = self.UaSp1(x3, x4)
        d4 = self.Up4(x4)
        d4 = torch.cat((x3, d4), dim=1)
        d4 = self.Up_conv4(d4)

        #d3 = self.UaSp2(x2, d4)
        d3 = self.Up3(d4)
        d3 = torch.cat((x2, d3), dim=1)
        d3 = self.Up_conv3(d3)

        #d2 = self.UaSp3(x1, d3)
        d2 = self.Up2(d3)
        d2 = torch.cat((x1, d2), dim=1)
        d2 = self.Up_conv2(d2)

        d1 = self.Conv_1x1(d2)
        #d1 = F.softmax(d1,dim=1)  # mine

        return [d1, d1,d1]

# if __name__ == '__main__':
#     net = U_Net(1,2)
#     in1 = torch.randn(1,1,64,64)
#     out1 = net(in1)
#     print(out1.shape)

# # 计算参数量
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)


# if __name__ == '__main__':
#     net = U_Net(1,2)
#     in1 = torch.randn(1,1,64,64)
#     out1 = net(in1)
#     # print(net)
#     print("Total number of parameters: " + str(count_parameters(net)))
#

# if __name__ == '__main__':
#     net = U_Net(1,2)
#     in1 = torch.randn(1,1,64,64)
#
# if __name__ == '__main__':
#     import time
#     batch_size = 1
#     # batch = torch.zeros([batch_size, 1, 80, 80], dtype=torch.float32)
#     batch = torch.randn(1,1,64,64)
#     model = net
#     print("Total params: {0:,}".format(sum(p.numel() for p in model.parameters() if p.requires_grad)))
#     print('Forward pass (bs={:d}) when running in the cpu:'.format(batch_size))
#     start_time = time.time()
#     logits = model(batch)
#     print("--- %s seconds ---" % (time.time() - start_time))
#
#     # 计算FLOPs & Params
#     from models.util import CalParams
#     t = CalParams(net.cuda(), torch.rand(1,1,64,64).cuda())
#     print(t)


# ============================================================
class R2U_Net(nn.Module):
    def __init__(self, img_ch=1, output_ch=2, t=2):
        super(R2U_Net, self).__init__()

        self.Maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Upsample = nn.Upsample(scale_factor=2)

        self.RRCNN1 = RRCNN_block(ch_in=img_ch, ch_out=64, t=t)
        self.ResPath1 = ResPath(input_channels=img_ch, output_channels=64,length=5)

        self.RRCNN2 = RRCNN_block(ch_in=64, ch_out=128, t=t)
        self.ResPath2 = ResPath(input_channels=64,output_channels=128,length=4)

        self.RRCNN3 = RRCNN_block(ch_in=128, ch_out=256, t=t)
        self.ResPath3 = ResPath(input_channels=128, output_channels=256,length=3)

        self.RRCNN4 = RRCNN_block(ch_in=256, ch_out=512, t=t)
        self.ResPath4 = ResPath(input_channels=256, output_channels=512,length=2)

        self.RRCNN5 = RRCNN_block(ch_in=512, ch_out=1024, t=t)
        self.ResPath5 = ResPath(input_channels=512,output_channels=1024,length=1)

        self.Up5 = up_conv(ch_in=1024, ch_out=512)
        self.Up_RRCNN5 = RRCNN_block(ch_in=1024, ch_out=512, t=t)

        self.Up4 = up_conv(ch_in=512, ch_out=256)
        self.Up_RRCNN4 = RRCNN_block(ch_in=512, ch_out=256, t=t)

        self.Up3 = up_conv(ch_in=256, ch_out=128)
        self.Up_RRCNN3 = RRCNN_block(ch_in=256, ch_out=128, t=t)

        self.Up2 = up_conv(ch_in=128, ch_out=64)
        self.Up_RRCNN2 = RRCNN_block(ch_in=128, ch_out=64, t=t)

        self.Conv_1x1 = nn.Conv2d(64, output_ch, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        # encoding path
        x1 = self.RRCNN1(x)
        # x1_1 = self.ResPath1(x1)
        #print(x1.shape)
        x2 = self.Maxpool(x1)
        x2 = self.RRCNN2(x2)
        # x2_1 = self.ResPath2(x2)

        x3 = self.Maxpool(x2)
        x3 = self.RRCNN3(x3)
        # x3_1 = self.ResPath3(x3)

        x4 = self.Maxpool(x3)
        x4 = self.RRCNN4(x4)
        # x4_1 = self.ResPath4(x4)

        x5 = self.Maxpool(x4)
        x5 = self.RRCNN5(x5)
        # x5_1 = self.ResPath5(x5)

        # decoding + concat path
        d5 = self.Up5(x5)
        d5 = torch.cat((x4, d5), dim=1)
        # d5 = torch.cat((x4_1, d5), dim=1)
        d5 = self.Up_RRCNN5(d5)

        d4 = self.Up4(d5)
        d4 = torch.cat((x3, d4), dim=1)
        # d4 = torch.cat((x3_1, d4), dim=1)
        d4 = self.Up_RRCNN4(d4)

        d3 = self.Up3(d4)
        d3 = torch.cat((x2, d3), dim=1)
        # d3 = torch.cat((x2_1, d3), dim=1)
        d3 = self.Up_RRCNN3(d3)

        d2 = self.Up2(d3)
        d2 = torch.cat((x1, d2), dim=1)
        # d2 = torch.cat((x1_1, d2), dim=1)
        d2 = self.Up_RRCNN2(d2)

        d1 = self.Conv_1x1(d2)
        d1 = F.softmax(d1,dim=1)

        return d1

# if __name__ == '__main__':
#     net = R2U_Net(1,2)
#     print(net)
#     in1 = torch.randn((64,1,48,48)).cuda()
#     out1 = net(in1)
#     print(out1.size())

# ===========================================================
class AttU_Net(nn.Module):
    def __init__(self, img_ch=3, output_ch=1):
        super(AttU_Net, self).__init__()

        self.Maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.Conv1 = conv_block(ch_in=img_ch, ch_out=64)
        self.Conv2 = conv_block(ch_in=64, ch_out=128)
        self.Conv3 = conv_block(ch_in=128, ch_out=256)
        self.Conv4 = conv_block(ch_in=256, ch_out=512)
        self.Conv5 = conv_block(ch_in=512, ch_out=1024)

        self.Up5 = up_conv(ch_in=1024, ch_out=512)
        self.Att5 = Attention_block(F_g=512, F_l=512, F_int=256)
        self.Up_conv5 = conv_block(ch_in=1024, ch_out=512)

        self.Up4 = up_conv(ch_in=512, ch_out=256)
        self.Att4 = Attention_block(F_g=256, F_l=256, F_int=128)
        self.Up_conv4 = conv_block(ch_in=512, ch_out=256)

        self.Up3 = up_conv(ch_in=256, ch_out=128)
        self.Att3 = Attention_block(F_g=128, F_l=128, F_int=64)
        self.Up_conv3 = conv_block(ch_in=256, ch_out=128)

        self.Up2 = up_conv(ch_in=128, ch_out=64)
        self.Att2 = Attention_block(F_g=64, F_l=64, F_int=32)
        self.Up_conv2 = conv_block(ch_in=128, ch_out=64)

        self.Conv_1x1 = nn.Conv2d(64, output_ch, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        # encoding path
        x1 = self.Conv1(x)

        x2 = self.Maxpool(x1)
        x2 = self.Conv2(x2)

        x3 = self.Maxpool(x2)
        x3 = self.Conv3(x3)

        x4 = self.Maxpool(x3)
        x4 = self.Conv4(x4)

        x5 = self.Maxpool(x4)
        x5 = self.Conv5(x5)

        # decoding + concat path
        d5 = self.Up5(x5)
        x4 = self.Att5(g=d5, x=x4)
        d5 = torch.cat((x4, d5), dim=1)
        d5 = self.Up_conv5(d5)

        d4 = self.Up4(d5)
        x3 = self.Att4(g=d4, x=x3)
        d4 = torch.cat((x3, d4), dim=1)
        d4 = self.Up_conv4(d4)

        d3 = self.Up3(d4)
        x2 = self.Att3(g=d3, x=x2)
        d3 = torch.cat((x2, d3), dim=1)
        d3 = self.Up_conv3(d3)

        d2 = self.Up2(d3)
        x1 = self.Att2(g=d2, x=x1)
        d2 = torch.cat((x1, d2), dim=1)
        d2 = self.Up_conv2(d2)

        d1 = self.Conv_1x1(d2)
        d1 = F.softmax(d1,dim=1)
        return d1


# if __name__ == '__main__':
#     net = AttU_Net(1,2)
#     in1 = torch.randn(1,1,64,64)
#
# if __name__ == '__main__':
#     import time
#     batch_size = 1
#     # batch = torch.zeros([batch_size, 1, 80, 80], dtype=torch.float32)
#     batch = torch.randn(1,1,64,64)
#     model = net
#     print("Total params: {0:,}".format(sum(p.numel() for p in model.parameters() if p.requires_grad)))
#     print('Forward pass (bs={:d}) when running in the cpu:'.format(batch_size))
#     start_time = time.time()
#     logits = model(batch)
#     print("--- %s seconds ---" % (time.time() - start_time))
#
#     # 计算FLOPs & Params
#     from models.util import CalParams
#     t = CalParams(net.cuda(), torch.rand(1,1,64,64).cuda())
#     print(t)

# ==============================================================
class R2AttU_Net(nn.Module):
    def __init__(self, img_ch=3, output_ch=1, t=2):
        super(R2AttU_Net, self).__init__()

        self.Maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Upsample = nn.Upsample(scale_factor=2)

        self.RRCNN1 = RRCNN_block(ch_in=img_ch, ch_out=64, t=t)

        self.RRCNN2 = RRCNN_block(ch_in=64, ch_out=128, t=t)

        self.RRCNN3 = RRCNN_block(ch_in=128, ch_out=256, t=t)

        self.RRCNN4 = RRCNN_block(ch_in=256, ch_out=512, t=t)

        self.RRCNN5 = RRCNN_block(ch_in=512, ch_out=1024, t=t)

        self.Up5 = up_conv(ch_in=1024, ch_out=512)
        self.Att5 = Attention_block(F_g=512, F_l=512, F_int=256)
        self.Up_RRCNN5 = RRCNN_block(ch_in=1024, ch_out=512, t=t)

        self.Up4 = up_conv(ch_in=512, ch_out=256)
        self.Att4 = Attention_block(F_g=256, F_l=256, F_int=128)
        self.Up_RRCNN4 = RRCNN_block(ch_in=512, ch_out=256, t=t)

        self.Up3 = up_conv(ch_in=256, ch_out=128)
        self.Att3 = Attention_block(F_g=128, F_l=128, F_int=64)
        self.Up_RRCNN3 = RRCNN_block(ch_in=256, ch_out=128, t=t)

        self.Up2 = up_conv(ch_in=128, ch_out=64)
        self.Att2 = Attention_block(F_g=64, F_l=64, F_int=32)
        self.Up_RRCNN2 = RRCNN_block(ch_in=128, ch_out=64, t=t)

        self.Conv_1x1 = nn.Conv2d(64, output_ch, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        # encoding path
        x1 = self.RRCNN1(x)

        x2 = self.Maxpool(x1)
        x2 = self.RRCNN2(x2)

        x3 = self.Maxpool(x2)
        x3 = self.RRCNN3(x3)

        x4 = self.Maxpool(x3)
        x4 = self.RRCNN4(x4)

        x5 = self.Maxpool(x4)
        x5 = self.RRCNN5(x5)

        # decoding + concat path
        d5 = self.Up5(x5)
        x4 = self.Att5(g=d5, x=x4)
        d5 = torch.cat((x4, d5), dim=1)
        d5 = self.Up_RRCNN5(d5)

        d4 = self.Up4(d5)
        x3 = self.Att4(g=d4, x=x3)
        d4 = torch.cat((x3, d4), dim=1)
        d4 = self.Up_RRCNN4(d4)

        d3 = self.Up3(d4)
        x2 = self.Att3(g=d3, x=x2)
        d3 = torch.cat((x2, d3), dim=1)
        d3 = self.Up_RRCNN3(d3)

        d2 = self.Up2(d3)
        x1 = self.Att2(g=d2, x=x1)
        d2 = torch.cat((x1, d2), dim=1)
        d2 = self.Up_RRCNN2(d2)

        d1 = self.Conv_1x1(d2)
        d1 = F.softmax(d1, dim=1)

        return d1

#==================DenseUNet=====================================
class Single_level_densenet(nn.Module):
    def __init__(self, filters, num_conv=4):
        super(Single_level_densenet, self).__init__()
        self.num_conv = num_conv
        self.conv_list = nn.ModuleList()
        self.bn_list = nn.ModuleList()
        for i in range(self.num_conv):
            self.conv_list.append(nn.Conv2d(filters, filters, 3, padding=1))
            self.bn_list.append(nn.BatchNorm2d(filters))

    def forward(self, x):
        outs = []
        outs.append(x)
        for i in range(self.num_conv):
            temp_out = self.conv_list[i](outs[i])
            if i > 0:
                for j in range(i):
                    temp_out += outs[j]
            outs.append(F.relu(self.bn_list[i](temp_out)))
        out_final = outs[-1]
        del outs
        return out_final


class Down_sample(nn.Module):
    def __init__(self, kernel_size=2, stride=2):
        super(Down_sample, self).__init__()
        self.down_sample_layer = nn.MaxPool2d(kernel_size, stride)

    def forward(self, x):
        y = self.down_sample_layer(x)
        return y, x


class Upsample_n_Concat(nn.Module):
    def __init__(self, filters):
        super(Upsample_n_Concat, self).__init__()
        self.upsample_layer = nn.ConvTranspose2d(filters, filters, 4, padding=1, stride=2)
        self.conv = nn.Conv2d(2 * filters, filters, 3, padding=1)
        self.bn = nn.BatchNorm2d(filters)

    def forward(self, x, y):
        x = self.upsample_layer(x)
        x = torch.cat([x, y], dim=1)
        x = F.relu(self.bn(self.conv(x)))
        return x


class Dense_Unet(nn.Module):
    def __init__(self, in_chan=3,out_chan=2,filters=128, num_conv=4):

        super(Dense_Unet, self).__init__()
        self.conv1 = nn.Conv2d(in_chan, filters, 1)
        self.d1 = Single_level_densenet(filters, num_conv)
        self.down1 = Down_sample()
        self.d2 = Single_level_densenet(filters, num_conv)
        self.down2 = Down_sample()
        self.d3 = Single_level_densenet(filters, num_conv)
        self.down3 = Down_sample()
        self.d4 = Single_level_densenet(filters, num_conv)
        self.down4 = Down_sample()
        self.bottom = Single_level_densenet(filters, num_conv)
        self.up4 = Upsample_n_Concat(filters)
        self.u4 = Single_level_densenet(filters, num_conv)
        self.up3 = Upsample_n_Concat(filters)
        self.u3 = Single_level_densenet(filters, num_conv)
        self.up2 = Upsample_n_Concat(filters)
        self.u2 = Single_level_densenet(filters, num_conv)
        self.up1 = Upsample_n_Concat(filters)
        self.u1 = Single_level_densenet(filters, num_conv)
        self.outconv = nn.Conv2d(filters, out_chan, 1)

    #         self.outconvp1 = nn.Conv2d(filters,out_chan, 1)
    #         self.outconvm1 = nn.Conv2d(filters,out_chan, 1)

    def forward(self, x):
        x = self.conv1(x)
        x, y1 = self.down1(self.d1(x))
        x, y2 = self.down1(self.d2(x))
        x, y3 = self.down1(self.d3(x))
        x, y4 = self.down1(self.d4(x))
        x = self.bottom(x)
        x = self.u4(self.up4(x, y4))
        x = self.u3(self.up3(x, y3))
        x = self.u2(self.up2(x, y2))
        x = self.u1(self.up1(x, y1))
        x1 = self.outconv(x)
        #         xm1 = self.outconvm1(x)
        #         xp1 = self.outconvp1(x)
        x1 = F.softmax(x1,dim=1)
        return x1
# =========================================================

if __name__ == '__main__':
    net = U_Net(img_ch=3, output_ch=1, fea_channels=64)


    from thop import profile
    input = torch.randn(1, 3, 480, 384).cuda().float()
    macs, params = profile(net.cuda(), inputs=(input,))
    print(macs, params)

    #print(net)
    in1 = torch.randn(4,3,224,224).cuda()
    out = net(in1)
    #print(out.size())

# if __name__ == '__main__':
#     # test network forward
#     net = AttU_Net(1,2).cuda()
#     print(net)
#     in1 = torch.randn((4,1,48,48)).cuda()
#     out1 = net(in1)
#     print(out1.size())


[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
184281661440.0 34527041.0


####dual_local_global_attention

In [16]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
# class LocalAttention(nn.Module):
#     def __init__(self, input_size, embed_size=10, win_size=5, out_channels=32):
#         super(LocalAttention, self).__init__()
#
#         self.input_size = input_size
#         self.embed_size = embed_size
#         self.win_size = win_size
#         self.out_channels = out_channels
#
#         self.attention_layer = nn.Sequential(
#             nn.Conv2d(1, 1, kernel_size=(self.win_size, self.embed_size)),
#             nn.Sigmoid())
#
#         self.cnn = nn.Sequential(
#             nn.Conv2d(1, self.out_channels, kernel_size=(1, self.embed_size)),
#             nn.Tanh(),
#             nn.MaxPool2d((self.input_size, 1)))
#
#     def forward(self, x):
#         padding = Variable(torch.zeros(x.size(0), (self.win_size - 1) // 2, self.embed_size))
#         padding = padding.cuda()
#         x_pad = torch.cat((padding, x, padding), 1)
#
#         x_pad = x_pad.unsqueeze(1)
#         scores = self.attention_layer(x_pad)
#
#         scores = scores.squeeze(1)
#
#         out = torch.mul(x, scores)
#
#         out = out.unsqueeze(1)
#         out = self.cnn(out)
#
#         return out
#
#
# class GlobalAttention(nn.Module):
#     def __init__(self, input_size, embed_size, out_channels):
#         super(GlobalAttention, self).__init__()
#
#         self.input_size = input_size
#         self.embed_size = embed_size
#         self.out_channels = out_channels
#
#         self.attention_layer = nn.Sequential(
#             nn.Conv2d(1, 1, kernel_size=(self.input_size, self.embed_size)),
#             nn.Sigmoid())
#
#         self.cnn_1 = nn.Sequential(
#             nn.Conv2d(1, self.out_channels, kernel_size=(2, self.embed_size)),
#             nn.Tanh(),
#             nn.MaxPool2d((self.input_size - 2 + 1, 1)))
#
#         self.cnn_2 = nn.Sequential(
#             nn.Conv2d(1, self.out_channels, kernel_size=(3, self.embed_size)),
#             nn.Tanh(),
#             nn.MaxPool2d((self.input_size - 3 + 1, 1)))
#
#         self.cnn_3 = nn.Sequential(
#             nn.Conv2d(1, self.out_channels, kernel_size=(4, self.embed_size)),
#             nn.Tanh(),
#             nn.MaxPool2d((self.input_size - 4 + 1, 1)))
#
#     def forward(self, x):
#         x = x.unsqueeze(1)
#         score = self.attention_layer(x)
#         out = torch.mul(x, score)
#         out_1 = self.cnn_1(out)
#         out_2 = self.cnn_2(out)
#         out_3 = self.cnn_3(out)
#         return (out_1, out_2, out_3)
#
#
# class CNNDLGA(nn.Module):
#
#     def __init__(self, input_size, embed_size=100, win_size=5, channels_local=200, channels_global=100,
#                  fc_input_size=500, hidden_size=500, output_size=50):
#         super(CNNDLGA, self).__init__()
#
#         self.localAttentionLayer_user = LocalAttention(input_size, embed_size, win_size, channels_local)
#         self.globalAttentionLayer_user = GlobalAttention(input_size, embed_size, channels_global)
#         self.localAttentionLayer_item = LocalAttention(input_size, embed_size, win_size, channels_local)
#         self.globalAttentionLayer_item = GlobalAttention(input_size, embed_size, channels_global)
#         self.fcLayer = nn.Sequential(
#             nn.Linear(fc_input_size, hidden_size),
#             nn.Dropout(0.5),
#             nn.ReLU(),
#             nn.Linear(hidden_size, output_size),
#         )
#
#     def forward(self, x_user, x_item):
#         # user
#         local_user = self.localAttentionLayer_user(x_user)
#         global1_user, global2_user, global3_user = self.globalAttentionLayer_user(x_user)
#         out_user = torch.cat((local_user, global1_user, global2_user, global3_user), 1)
#         out_user = out_user.view(out_user.size(0), -1)
#         out_user = self.fcLayer(out_user)
#
#         # item
#         local_item = self.localAttentionLayer_item(x_item)
#         global1_item, global2_item, global3_item = self.globalAttentionLayer_item(x_item)
#         out_item = torch.cat((local_item, global1_item, global2_item, global3_item), 1)
#         out_item = out_item.view(out_item.size(0), -1)
#         out_item = self.fcLayer(out_item)
#
#         out = torch.sum(torch.mul(out_user, out_item), 1)
#
#         return out


class Patch_AttentionV2(nn.Module):
    def __init__(self, in_channels, reduction=2, pool_window=5, add_input=False):
        super(Patch_AttentionV2, self).__init__()
        self.pool_window = pool_window
        self.add_input = add_input
        self.SA = nn.Sequential(
            nn.AvgPool2d(kernel_size=pool_window + 1, stride=1, padding=pool_window // 2),
            nn.Conv2d(in_channels, in_channels // reduction, 1),
            nn.BatchNorm2d(in_channels // reduction, momentum=0.95),
            nn.ReLU(inplace=False),
            nn.Conv2d(in_channels // reduction, in_channels, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, h, w = x.size()
        A = self.SA(x)

        A = F.upsample(A, (h, w), mode='bilinear')
        output = x * A
        if self.add_input:
            output += x

        return output

####unetrpp_ff

In [17]:
import torch
from torch import nn
from torch.nn import functional as F
from typing import Tuple, Union
#from network_architecture.neural_network import SegmentationNetwork
#from network_architecture.dynunet_block import UnetOutBlock, UnetResBlock
#from network_architecture.synapse.model_components import UnetrPPEncoder, UnetrUpBlock
#from  net.UNetFamily import U_Nett
#from net.dual_local_global_attention import Patch_AttentionV2
class UpSampling(nn.Module):
    def __init__(self, scale_factor =2):
        super(UpSampling, self).__init__()
        self.upsample = nn.Upsample(scale_factor = scale_factor,  mode='bilinear', align_corners=None)

    def forward(self, x):
        x = self.upsample(x)
        return x

class FeatureFusion(nn.Module):
    def __init__(self, in_channels, out_channels, norm_name='batch'):
        super(FeatureFusion, self).__init__()
        self.urb1 = UnetResBlock(
            spatial_dims=2,
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=3,
            stride=1,
            norm_name=norm_name,
        )
        self.conv1 = nn.Conv2d(out_channels, out_channels, kernel_size=5, stride=3, padding=2, bias=False)
        self.conv2 = nn.Conv2d(out_channels*2, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.urb2 = UnetResBlock(
            spatial_dims=2,
            in_channels=out_channels*2,
            out_channels=out_channels*1,
            kernel_size=3,
            stride=1,
            norm_name=norm_name,
        )


    def forward(self, x, multif):
        b, c, h, w = x.shape
        dec1, dec2 = multif
        urb1 = self.urb1(x)

        conv1 = self.conv1(urb1)
        dec1 = self.conv2(dec1)
        conv1 = torch.cat([conv1, dec1], dim=1)
        urb2 = self.urb2(conv1)



        urb2 = F.interpolate(urb2, (h, w ), mode='bilinear')

        ff = torch.cat([urb1, urb2], dim=1)

        return ff



class RetinalVasularSegFF(SegmentationNetwork):
    def __init__(
            self,
            in_channels: int,
            out_channels: int,
            img_size: [640, 640],
            feature_size: int = 16,
            hidden_size: int = 256,
            num_heads: int = 4,
            pos_embed: str = "perceptron",  # TODO: Remove the argument
            norm_name: Union[Tuple, str] = "instance",
            dropout_rate: float = 0.0,
            depths=None,
            dims=None,
            conv_op=nn.Conv3d,
            do_ds=True,

    ) -> None:
        super().__init__()
        if depths is None:
            depths = [3, 3, 3, 3]
        self.do_ds = do_ds
        self.conv_op = conv_op
        self.num_classes = out_channels
        if not (0 <= dropout_rate <= 1):
            raise AssertionError("dropout_rate should be between 0 and 1.")

        if pos_embed not in ["conv", "perceptron"]:
            raise KeyError(f"Position embedding layer of type {pos_embed} is not supported.")

        self.patch_size = (3, 3)
        self.feat_size = (
            img_size[0] // self.patch_size[0] // 8,  # 8 is the downsampling happened through the four encoders stages
            img_size[1] // self.patch_size[1] // 8,  # 8 is the downsampling happened through the four encoders stages
        )
        self.hidden_size = hidden_size
        self.input_size = [self.feat_size[0]*8*self.feat_size[1]*8, self.feat_size[0]*4*self.feat_size[1]*4,
                           self.feat_size[0]*2*self.feat_size[1]*2, self.feat_size[0]*1*self.feat_size[1]*1]
        self.unetr_pp_encoder = UnetrPPEncoder(input_size=self.input_size, patch_size=self.patch_size, dims=dims, depths=depths, num_heads=num_heads, in_channels=feature_size)

        self.prarm = nn.Parameter(torch.tensor([0.2, 0.6, 0.2]),requires_grad=True).cuda().float()
        norm_name ='batch'
        self.unet = U_Nett(img_ch=feature_size, output_ch=feature_size, fea_channels=32)
        self.encoder1 = UnetResBlock(
            spatial_dims=2,
            in_channels=in_channels,
            out_channels=feature_size,
            kernel_size=3,
            stride=1,
            norm_name=norm_name,
        )

        self.la1 = UnetResBlock(
            spatial_dims=2,
            in_channels=feature_size*2,
            out_channels=feature_size*2,
            kernel_size=3,
            stride=1,
            norm_name=norm_name,
        )
        self.la2 = UnetResBlock(
            spatial_dims=2,
            in_channels=feature_size*4,
            out_channels=feature_size*4,
            kernel_size=3,
            stride=1,
            norm_name=norm_name,
        )
        self.la3 = UnetResBlock(
            spatial_dims=2,
            in_channels=feature_size*8,
            out_channels=feature_size*8,
            kernel_size=3,
            stride=1,
            norm_name=norm_name,
        )



        self.decoder5 = UnetrUpBlock(
            spatial_dims=2,
            in_channels=feature_size * 16,
            out_channels=feature_size * 8,
            kernel_size=3,
            upsample_kernel_size=2,
            norm_name=norm_name,
            out_size=img_size[0]//(self.patch_size[0]*4) * img_size[1]//(self.patch_size[1]*4),
        )
        self.decoder4 = UnetrUpBlock(
            spatial_dims=2,
            in_channels=feature_size * 8,
            out_channels=feature_size * 4,
            kernel_size=3,
            upsample_kernel_size=2,
            norm_name=norm_name,
            out_size=img_size[0]//(self.patch_size[0]*2) * img_size[1]//(self.patch_size[1]*2),
        )
        self.decoder3 = UnetrUpBlock(
            spatial_dims=2,
            in_channels=feature_size * 4,
            out_channels=feature_size * 2,
            kernel_size=3,
            upsample_kernel_size=2,
            norm_name=norm_name,
            out_size=img_size[0]//self.patch_size[0] * img_size[1]//self.patch_size[1],
        )
        self.decoder2 = UnetrUpBlock(
            spatial_dims=2,
            in_channels=feature_size * 2,
            out_channels=feature_size,
            kernel_size=3,
            upsample_kernel_size=self.patch_size,
            norm_name=norm_name,
            out_size=img_size[0] * img_size[1],
            conv_decoder=True,
        )
        # self.FeaFus = FeatureFusion(feature_size, feature_size)
        self.out1 = UnetOutBlock(spatial_dims=2, in_channels=feature_size, out_channels=out_channels)
        # if self.do_ds:
        #     self.out2 = UnetOutBlock(spatial_dims=2, in_channels=feature_size * 2, out_channels=out_channels)
        #     self.out3 = UnetOutBlock(spatial_dims=2, in_channels=feature_size * 4, out_channels=out_channels)

    def proj_feat(self, x, hidden_size, feat_size):
        x = x.view(x.size(0), feat_size[0], feat_size[1], hidden_size)
        x = x.permute(0, 3, 1, 2).contiguous()
        return x

    def forward(self, x_in):

        convBlock = self.encoder1(x_in)
        x_output, hidden_states = self.unetr_pp_encoder(convBlock)


        # convBlock = self.encoder11(convBlock)
        convBlock1 = self.unet(convBlock)[0]

        # Four encoders
        enc1 = hidden_states[0]
        enc2 = hidden_states[1]
        enc3 = hidden_states[2]
        enc4 = hidden_states[3]

        enc1 = self.la1(enc1)
        enc2 = self.la2(enc2)
        enc3 = self.la3(enc3)

        # Four decoders
        dec4 = self.proj_feat(enc4, self.hidden_size, self.feat_size)
        dec3 = self.decoder5(dec4, enc3)
        dec2 = self.decoder4(dec3, enc2)
        dec1 = self.decoder3(dec2, enc1)

        decoder2 = self.decoder2(dec1, convBlock1)
        out = decoder2#self.FeaFus(decoder2, [dec1, dec2])
        # if self.do_ds:
        #     logits = [self.out1(out), self.out2(dec1), self.out3(dec2)]
        # else:
        logits = self.out1(out)

        return logits, logits, logits

####loss

In [18]:
import cv2
import numpy as np
import torch
from torch.nn import functional as F
#import config.config as cfg

def dice_loss(score, target):
    target = target.float()
    smooth = 1e-5
    intersect = torch.sum(score * target)
    y_sum = torch.sum(target * target)
    z_sum = torch.sum(score * score)
    loss = (2 * intersect + smooth) / (z_sum + y_sum + smooth)
    loss = 1 - loss
    return loss


def secondary_losses(pred, train_label):
    train_label = torch.unsqueeze(train_label, dim=1)
    depth, height = cfg.IMG_SIZE//3, cfg.IMG_SIZE//3
    train_label = F.interpolate(train_label, (depth, height), mode='nearest')
    train_label = torch.squeeze(train_label, dim=1)


    loss_seg = F.cross_entropy(pred, train_label.long())
    outputs_soft = F.softmax(pred, dim=1)
    loss_dice_ = dice_loss(outputs_soft[:, 1, :, :], train_label[:, :, :])
    loss = 1.0 * (loss_dice_ + loss_seg)
    return  loss



class FocalLoss2d(torch.nn.Module):
    def __init__(self, alpha=0.25, gamma=2, size_average=True, ignore_index=255):
        super(FocalLoss2d, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.size_average = size_average # 对batch里面的数据取均值/求和

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets.long(), reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1-pt)**self.gamma * ce_loss
        if self.size_average:
            return focal_loss.mean()
        else:
            return focal_loss.sum()

class WeightedFocalLoss(torch.nn.Module):
    "Non weighted version of Focal Loss"
    def __init__(self, alpha=.25, gamma=2):
        super(WeightedFocalLoss, self).__init__()
        self.alpha = torch.tensor([alpha, 1-alpha]).cuda()
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none').view(-1)#

        targets = targets.type(torch.long)
        at = self.alpha.gather(0, targets.data.view(-1))
        pt = torch.exp(-BCE_loss)
        F_loss = at*(1-pt)**self.gamma * BCE_loss
        return F_loss.mean()



def cal_sen(outputs_soft, train_label):

    bn, h, w = outputs_soft.shape
    Sen = 0
    Acc = 0
    Spec = 0
    nm = 0
    for i in range(bn):
        b_data = torch.gt(outputs_soft[i], 0.5).float()#[ph: ph+height, pw:pw+width], 0.5).float()
        b_label = train_label[i]#[ph: ph+height, pw:pw+width].float()
        TP = torch.sum(b_data*b_label)
        FN = torch.sum((1 - b_data) *b_label)

        TN = torch.sum((1 - b_data) * (1- b_label))
        FP = torch.sum((b_data) * (1- b_label))

        if (TP + FN) ==0:
            Sen = Sen + 0
            nm = nm +1
        else:
            Sen = Sen + TP / (TP + FN)


        Acc = Acc + (TP + TN)/(TP + FN + TN + FP)
        Spec = Spec + (TN)/(TN + FP)

    Sen = Sen / (bn-nm)
    Acc = Acc / bn
    Spec = Spec / bn

    return Sen, Acc, Spec










##metrics

In [19]:
"""
This part contains functions related to the calculation of performance indicators  介绍性能指标计算及相关功能
"""
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
import os
import torch
from os.path import join
import numpy as np
from collections import OrderedDict
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
params = {'legend.fontsize': 13,
         'axes.labelsize': 15,
         'axes.titlesize':15,
         'xtick.labelsize':15,
         'ytick.labelsize':15} # define pyplot parameters
pylab.rcParams.update(params)
#Area under the ROC curve

class Evaluate():
    def __init__(self,save_path=None):
        self.target = None
        self.output = None
        self.save_path = save_path
        if self.save_path is not None:
            if not os.path.exists(self.save_path):
                os.makedirs(self.save_path)
        self.threshold_confusion = 0.5

    # Add data pair (target and predicted value)
    def add_batch(self,batch_tar,batch_out):
        batch_tar = batch_tar.flatten()
        batch_out = batch_out.flatten()

        self.target = batch_tar if self.target is None else np.concatenate((self.target,batch_tar))
        self.output = batch_out if self.output is None else np.concatenate((self.output,batch_out))

    # Plot ROC and calculate AUC of ROC
    def auc_roc(self,plot=False):
        AUC_ROC = roc_auc_score(self.target, self.output)
        # print("\nAUC of ROC curve: " + str(AUC_ROC))
        if plot and self.save_path is not None:
            fpr, tpr, thresholds = roc_curve(self.target, self.output)
            # print("\nArea under the ROC curve: " + str(AUC_ROC))
            plt.figure()
            plt.plot(fpr, tpr, '-', label='Area Under the Curve (AUC = %0.4f)' % AUC_ROC)
            plt.title('ROC curve')
            plt.xlabel("FPR (False Positive Rate)")
            plt.ylabel("TPR (True Positive Rate)")
            plt.legend(loc="lower right")
            plt.savefig(join(self.save_path , "ROC.png"))
        return AUC_ROC

    # Plot PR curve and calculate AUC of PR curve
    def auc_pr(self,plot=False):
        precision, recall, thresholds = precision_recall_curve(self.target, self.output)
        precision = np.fliplr([precision])[0]
        recall = np.fliplr([recall])[0]
        AUC_pr = np.trapz(precision, recall)
        # print("\nAUC of P-R curve: " + str(AUC_pr))
        if plot and self.save_path is not None:

            plt.figure()
            plt.plot(recall, precision, '-', label='Area Under the Curve (AUC = %0.4f)' % AUC_pr)
            plt.title('Precision - Recall curve')
            plt.xlabel("Recall")
            plt.ylabel("Precision")
            plt.legend(loc="lower right")
            plt.savefig(join(self.save_path ,"Precision_recall.png"))
        return AUC_pr

    # Accuracy, specificity, sensitivity, precision can be obtained by calculating the confusion matrix
    def confusion_matrix(self):
        #Confusion matrix
        y_pred = self.output>=self.threshold_confusion
        confusion = confusion_matrix(self.target, y_pred)
        # print(confusion)
        accuracy = 0
        if float(np.sum(confusion))!=0:
            accuracy = float(confusion[0,0]+confusion[1,1])/float(np.sum(confusion))
        # print("Global Accuracy: " +str(accuracy))
        specificity = 0
        if float(confusion[0,0]+confusion[0,1])!=0:
            specificity = float(confusion[0,0])/float(confusion[0,0]+confusion[0,1])
        # print("Specificity: " +str(specificity))
        sensitivity = 0
        if float(confusion[1,1]+confusion[1,0])!=0:
            sensitivity = float(confusion[1,1])/float(confusion[1,1]+confusion[1,0])
        # print("Sensitivity: " +str(sensitivity))
        precision = 0
        if float(confusion[1,1]+confusion[0,1])!=0:
            precision = float(confusion[1,1])/float(confusion[1,1]+confusion[0,1])
        # print("Precision: " +str(precision))
        return confusion,accuracy,specificity,sensitivity,precision

    # Jaccard similarity index
    def jaccard_index(self):
        pass
        # jaccard_index = jaccard_similarity_score(y_true, y_pred, normalize=True)
        # print("\nJaccard similarity score: " +str(jaccard_index))

    # calculating f1_score
    def f1_score(self):
        pred = self.output>=self.threshold_confusion
        F1_score = f1_score(self.target, pred, labels=None, average='binary', sample_weight=None)
        # print("F1 score (F-measure): " +str(F1_score))
        return F1_score

    # Save performance results to specified file
    def save_all_result(self,plot_curve=True,save_name=None):
        #Save the results
        AUC_ROC = self.auc_roc(plot=plot_curve)
        AUC_pr  = self.auc_pr(plot=plot_curve)
        F1_score = self.f1_score()
        confusion,accuracy, specificity, sensitivity, precision = self.confusion_matrix()
        if save_name is not None:
            file_perf = open(join(self.save_path, save_name), 'w')
            file_perf.write("AUC ROC curve: "+str(AUC_ROC)
                            + "\nAUC PR curve: " +str(AUC_pr)
                            # + "\nJaccard similarity score: " +str(jaccard_index)
                            + "\nF1 score: " +str(F1_score)
                            +"\nAccuracy: " +str(accuracy)
                            +"\nSensitivity(SE): " +str(sensitivity)
                            +"\nSpecificity(SP): " +str(specificity)
                            +"\nPrecision: " +str(precision)
                            + "\n\nConfusion matrix:"
                            + str(confusion)
                            )
            file_perf.close()
        return OrderedDict([("AUC_ROC",AUC_ROC),("AUC_PR",AUC_pr),
                            ("f1-score",F1_score),("Acc",accuracy),
                            ("SE",sensitivity),("SP",specificity),
                            ("precision",precision)
                            ])

##model forward

In [25]:
from __future__ import print_function
import os
import time
import argparse
import cv2
import math
import torch
import torch.nn as nn
from torch.cuda.amp import autocast, GradScaler
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR
#from data.load_train_data import TrainData
#from data.load_test_data import TestData
from torch.utils.data import DataLoader
import sklearn.metrics as metrics
#from net.retinal_vasuclar_net import RetinalVasularSeg
#from net.unetrpp_ff import RetinalVasularSegFF
#from net.UNetFamily import U_Net, AttU_Net, Dense_Unet
#from net.loss import dice_loss, cal_sen
#from metrics import Evaluate
#import config.config as cfg

def model_forward(model, test_data, patch_size, hh,ww, stride_y, stride_x):

    sy = math.ceil((hh - patch_size[0]) / stride_y) + 1
    sx = math.ceil((ww - patch_size[1]) / stride_x) + 1

    score_map = np.zeros((1, hh, ww)).astype(np.float32)
    cnt = np.zeros((1, hh, ww)).astype(np.int32)
    test_datas = []
    boxes = []
    for y in range(0, sy):
        ys = min(stride_y * y, hh - patch_size[0])
        for x in range(0, sx):
            xs = min(stride_x * x, ww - patch_size[1])

            test_patch = test_data[:, :, ys:ys + patch_size[0], xs:xs + patch_size[1]]
            test_datas.append(test_patch)
            boxes.append([ys, ys + patch_size[0], xs, xs + patch_size[1]])

    bsize = 8
    batch_nums = math.ceil(len(test_datas)/bsize)
    for i in range(batch_nums):
        test_patch = torch.cat(test_datas[i*bsize:(i+1)*bsize], dim=0)
        outputs_segb = model(test_patch)[0]
        outputs_softb = F.sigmoid(outputs_segb)
        # outputs_softb = torch.softmax(outputs_segb, dim=1)[:, 1, :, :]
        predb = torch.squeeze(outputs_softb).detach().cpu().numpy()

        for j in range(predb.shape[0]):
            y1, y2, x1, x2 = boxes[i*bsize +j]
            score_map[0, y1: y2, x1: x2] = score_map[0, y1: y2, x1: x2] + predb[j]

            cnt[0, y1: y2, x1: x2] = cnt[0, y1: y2, x1: x2] + 1

    # cv2.namedWindow("score_map", cv2.WINDOW_NORMAL)
    # cv2.imshow("score_map", score_map[0, :, :])
    #
    # cv2.waitKey(0)
    score_map = torch.tensor(score_map / cnt).cuda().float()

    return  score_map


##main ##This part you uploaded after removing the extra parts and I was able to run it with a small change in "args". You can see the execution results and errors.

In [26]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import print_function
import os
import time
import argparse
import torch
import torch.nn as nn
from torch.cuda.amp import autocast, GradScaler
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR
#from data.load_train_data import TrainData, data_crop
#from data.load_test_data import TestData
from torch.utils.data import DataLoader
import sklearn.metrics as metrics
#from net.retinal_vasuclar_net import RetinalVasularSeg
#from net.unetrpp_ff import RetinalVasularSegFF
#from net.UNetFamily import U_Net, AttU_Net, Dense_Unet
# from net.Hessian import HessianNet
#from net.loss import dice_loss, FocalLoss2d, WeightedFocalLoss, cal_sen
#from test import model_forward
#import config.config as cfg

#def model_initial(model, model_name):
#    # 加载预训练模型
#    pretrained_dict = torch.load(model_name)["model"]
#    model_dict = model.state_dict()
#    # 1. filter out unnecessary keys
#    # pretrained_dictf = {k.replace('module.', ""): v for k, v in pretrained_dict.items() if k.replace('module.', "") in model_dict}
#    pretrained_dictf = {k: v for k, v in pretrained_dict.items() if k in model_dict}
#    # 2. overwrite entries in the existing state dict
#    model_dict.update(pretrained_dictf)
#    # 3. load the new state dict
#    model.load_state_dict(model_dict)

#    print("over")


def _init_():
    if not os.path.exists('outputs'):
        os.makedirs('outputs')
    if not os.path.exists('./outputs/' + args.exp_name):
        os.makedirs('./outputs/' + args.exp_name)
    if not os.path.exists('./outputs/' + args.exp_name + '/' + 'models'):
        os.makedirs('./outputs/' + args.exp_name + '/' + 'models')
    os.system('cp main_cls.py outputs' + '/' + args.exp_name + '/' + 'main_cls.py.backup')
    os.system('cp model.py outputs' + '/' + args.exp_name + '/' + 'model.py.backup')
    os.system('cp util.py outputs' + '/' + args.exp_name + '/' + 'util.py.backup')
    os.system('cp data.py outputs' + '/' + args.exp_name + '/' + 'data.py.backup')

class IOStream():
    def __init__(self, path):
        self.f = open(path, 'a')

    def cprint(self, text):
        print(text)
        self.f.write(text+'\n')
        self.f.flush()

    def close(self):
        self.f.close()

def train(args, io):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    file_path = "/content/training/images/"
    label_path = "/content/training/1st_manual/"
    #file_path = "F:/DRIVE/training/images/"
    #label_path = "F:/DRIVE/training/1st_manual/"
    # file_path = "E:/vasular/CHASEDB1/images/"
    # label_path = "E:/vasular/CHASEDB1/1st_label/"
    # file_path = "G:/vasular/STAREdatabase/images/"
    # label_path = "G:/vasular/STAREdatabase/labels-ah/"
    # file_path = "G:/vasular/HRFdatas/images/"
    # label_path = "G:/vasular/HRFdatas/images/"
    train_loader = DataLoader(TrainData(file_path, label_path, train_flag = True), num_workers=0,
                              batch_size=args.batch_size, shuffle=True, drop_last=True)

    file_path = "/content/test/images/"
    label_path = "/content/test/1st_manual/"
    #file_path = "F:/DRIVE/test/images/"
    #label_path = "F:/DRIVE/test/1st_manual/"
    # file_path = "E:/vasular/CHASEDB1/test/"
    # label_path = "E:/vasular/CHASEDB1/1st_label/"
    # file_path = "G:/vasular/STAREdatabase/test/"
    # label_path = "G:/vasular/STAREdatabase/labels-ah/"
    # file_path = "G:/vasular/HRFdatas/test/"
    # label_path = "G:/vasular/HRFdatas/test/"
    test_loader = DataLoader(TestData(file_path, label_path, train_flag = False), num_workers=0,
                             batch_size=args.test_batch_size, shuffle=True, drop_last=False)

    # Try to load models
    model = RetinalVasularSegFF(in_channels=3,
                            out_channels=2,
                            img_size=[192, 192],
                            feature_size=16,
                            num_heads=4,
                            depths=[3, 3, 3, 3],
                            dims=[32, 64, 128, 256],
                              hidden_size=256,
                            do_ds=True,
                            )

    #model_name = "./outputs/teethseg_model_200.pth"
    # model_initial(model, model_name)

    opt = optim.Adam(model.parameters(), lr=args.lr, weight_decay=1e-4)
    scheduler = CosineAnnealingLR(opt, args.epochs, eta_min=1e-6, last_epoch = -1)

    focalLoss = FocalLoss2d(gamma=2)#WeightedFocalLoss()#

    model.cuda()
    model.train()
    scaler = GradScaler()
    inter_nums = len(train_loader)
    total_acc = 0
    for epoch in range(0, args.epochs):
        ####################
        # Train
        ####################

        if args.scheduler == 'cos':
            scheduler.step()
        elif args.scheduler == 'step':
            if opt.param_groups[0]['lr'] > 1e-5:
                scheduler.step()
            if opt.param_groups[0]['lr'] < 1e-5:
                for param_group in opt.param_groups:
                    param_group['lr'] = 1e-5

        train_loss = 0.0
        loss_dice = 0
        sen_v = 0
        acc_v = 0
        spec_v = 0
        # for data, edges, label in train_loader:
        tic = time.time()
        nums = 0
        model.train()
        for train_data, train_label, weight_ in train_loader:

            train_data, train_label, train_weight_ = data_crop(train_data, train_label, weight_)

            train_data = train_data.cuda().float()
            train_label = train_label.cuda().float()
            train_weight_ = train_weight_.cuda().float()

            nums = nums +1
            opt.zero_grad()
            with autocast():
                outputs_seg, out2, _ = model(train_data)

                train_label[train_label >= 0.5] = 1
                loss_seg = F.cross_entropy(outputs_seg, train_label.long())
                # loss_seg = F.binary_cross_entropy_with_logits(outputs_seg[:,0, :, :], train_label)
                sec_loss = focalLoss(outputs_seg, train_label[:, :, :].long())

                outputs_soft = torch.softmax(outputs_seg, dim=1)[:, 1, :, :]#F.sigmoid(outputs_seg[:, 0, :, :])#
                loss_dice_ = dice_loss(outputs_soft, train_label[:, :, :])

                sen_v_, acc_v_, spec_v_ = cal_sen(outputs_soft, train_label[:, :, :].long())

                loss = (loss_seg + 1*loss_dice_ + 2*sec_loss)*1  #

            scaler.scale(loss).backward()
            # Unscales gradients and calls
            # or skips optimizer.step()
            scaler.step(opt)
            # Updates the scale for next iteration
            scaler.update()

            train_loss += loss.item()
            loss_dice += loss_dice_.item()
            sen_v += sen_v_.item()
            acc_v += acc_v_.item()
            spec_v += spec_v_.item()
            VIEW_NUMS = 5
            if nums % VIEW_NUMS == 0:
                toc = time.time()
                train_loss = train_loss/ (VIEW_NUMS)
                loss_dice = loss_dice / (VIEW_NUMS)
                sen_v = sen_v/(VIEW_NUMS)
                acc_v = acc_v/(VIEW_NUMS)
                spec_v = spec_v/(VIEW_NUMS)

                print("lr = ", opt.param_groups[0]['lr'])
                outstr = 'epoch %d /%d,epoch %d /%d, loss: %.6f, loss_dice: %.6f, sen_v: %.6f, acc_v: %.6f, spec_v: %.6f, const time: %.6f' % (
                 epoch,args.epochs, nums, inter_nums, train_loss, loss_dice, sen_v, acc_v, spec_v, toc - tic)

                io.cprint(outstr)
                train_loss = 0.0
                loss_dice = 0
                sen_v = 0
                acc_v = 0
                spec_v = 0
                tic = time.time()
        if 0 == epoch % 10 and epoch>300:
            test_nums = 0
            loss_dice, sen_v, acc_v, spec_v =0, 0, 0, 0
            model.eval()
            CROP_SIZE = 192
            patch_size = [CROP_SIZE, CROP_SIZE]
            stride_y, stride_x = 128, 128
            for test_data, test_label in test_loader:
                test_data = test_data.cuda().float()
                test_label = test_label.cuda().float()
                nums = nums + 1
                test_nums = test_nums + 1
                hh, ww = test_label.shape[-2:]
                with autocast():
                    score_map = model_forward(model, test_data, patch_size, hh, ww, stride_y, stride_x)
                    train_label[train_label > 0.5] = 1
                    loss_dice_ = dice_loss(score_map, test_label.long())
                    sen_v_, acc_v_, spec_v_ = cal_sen(score_map.detach(), test_label.detach().long())
                loss = 1.0 * (loss_dice_).item()
                sen_v = sen_v + sen_v_.item()
                acc_v = acc_v + acc_v_.item()
                spec_v = spec_v + spec_v_.item()
                loss_dice = loss_dice + loss_dice_.item()
            sen_v = sen_v/test_nums
            acc_v = acc_v/test_nums
            spec_v = spec_v/test_nums
            loss_dice = loss_dice/test_nums
            toc = time.time()
            outstr = 'epoch %d, loss: %.6f, loss_dice: %.6f, sen_v: %.6f, acc_v: %.6f, spec_v: %.6f, const time: %.6f' % (
                epoch, loss, loss_dice, sen_v, acc_v, spec_v, toc - tic)
            io.cprint("test   "+outstr)
            if (total_acc < (sen_v*0.35+acc_v*0.35+spec_v*0.35)):
                total_acc = (sen_v * 0.35 + acc_v * 0.35 + spec_v * 0.35)
                torch.save({'model': model.state_dict(), 'epoch': epoch},
                           'outputs/' + str(sen_v)+"_"+ str(acc_v)+ "_"+str(spec_v) + '.pth')

        if (epoch) % 200 == 0:
            torch.save({'model': model.state_dict(), 'epoch': epoch}, 'outputs/teethseg_model_' + str(epoch)+ '.pth')


if __name__ == "__main__":


    torch.backends.cudnn.enabled = True
    # Training settings
    parser = argparse.ArgumentParser(description='retinal vascular segmentation')
    parser.add_argument('--exp_name', type=str, default='retinal', metavar='N',
                        help='Name of the experiment')
    parser.add_argument('--batch_size', type=int, default=4, metavar='batch_size',
                        help='Size of batch)')
    parser.add_argument('--test_batch_size', type=int, default=1, metavar='batch_size',
                        help='Size of batch)')
    parser.add_argument('--epochs', type=int, default=1001, metavar='N',
                        help='number of episode to train ')
    parser.add_argument('--lr', type=float, default=2*1e-3, metavar='LR',
                        help='learning rate (default: 0.001, 0.1 if using sgd)')
    parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                        help='SGD momentum (default: 0.9)')
    parser.add_argument('--scheduler', type=str, default='cos', metavar='N',
                        choices=['cos', 'step'],
                        help='Scheduler to use, [cos, step]')
    parser.add_argument('--no_cuda', type=bool, default=False,
                        help='enables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    #args = parser.parse_args()
    args, unknown = parser.parse_known_args()
    _init_()

    torch.cuda.manual_seed(args.seed)
    io = IOStream('outputs/' + args.exp_name + '/run.log')
    io.cprint(str(args))

    train(args, io)



Namespace(exp_name='retinal', batch_size=4, test_batch_size=1, epochs=1001, lr=0.002, momentum=0.9, scheduler='cos', no_cuda=False, seed=1)
lr =  0.001999995077519133
epoch 0 /1001,epoch 5 /5, loss: 1.491050, loss_dice: 0.709165, sen_v: 0.428682, acc_v: 0.570354, spec_v: 0.589697, const time: 3.778917
lr =  0.001999980310125019
epoch 1 /1001,epoch 5 /5, loss: 1.182309, loss_dice: 0.672420, sen_v: 0.000049, acc_v: 0.883301, spec_v: 0.999996, const time: 3.488863
lr =  0.001999955697963115
epoch 2 /1001,epoch 5 /5, loss: 1.054867, loss_dice: 0.599031, sen_v: 0.000031, acc_v: 0.876826, spec_v: 0.999996, const time: 3.763355
lr =  0.0019999212412758474
epoch 3 /1001,epoch 5 /5, loss: 0.863069, loss_dice: 0.501134, sen_v: 0.202521, acc_v: 0.900608, spec_v: 0.990257, const time: 3.689625
lr =  0.001999876940402611
epoch 4 /1001,epoch 5 /5, loss: 0.710403, loss_dice: 0.401850, sen_v: 0.493898, acc_v: 0.908211, spec_v: 0.960766, const time: 3.429961
lr =  0.001999822795779764
epoch 5 /1001,epo

ValueError: could not broadcast input array from shape (2,192,192) into shape (192,192)

##main# This part is the last series of code in the "main" part

In [23]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import print_function
import os
import time
import argparse
import torch
import torch.nn as nn
from torch.cuda.amp import autocast, GradScaler
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR
#from data.load_train_data import TrainData, data_crop
#from data.load_test_data import TestData
from torch.utils.data import DataLoader
import sklearn.metrics as metrics
#from net.retinal_vasuclar_net import RetinalVasularSeg
#from net.unetrpp_ff import RetinalVasularSegFF
#from net.UNetFamily import U_Net, AttU_Net, Dense_Unet
# #from net.Hessian import HessianNet
#from net.loss import dice_loss, FocalLoss2d, WeightedFocalLoss, cal_sen
#from test import model_forward
#import config.config as cfg

def model_initial(model, model_name):
    # 加载预训练模型
    pretrained_dict = torch.load(model_name)["model"]
    model_dict = model.state_dict()
    # 1. filter out unnecessary keys
    # pretrained_dictf = {k.replace('module.', ""): v for k, v in pretrained_dict.items() if k.replace('module.', "") in model_dict}
    pretrained_dictf = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    # 2. overwrite entries in the existing state dict
    model_dict.update(pretrained_dictf)
    # 3. load the new state dict
    model.load_state_dict(model_dict)

    print("over")


def _init_(exp_name):
    if not os.path.exists('outputs'):
        os.makedirs('outputs')
    if not os.path.exists('./outputs/' + exp_name):
        os.makedirs('./outputs/' + exp_name)
    if not os.path.exists('./outputs/' + exp_name + '/' + 'models'):
        os.makedirs('./outputs/' + exp_name + '/' + 'models')
    os.system('cp main_cls.py outputs' + '/' + exp_name + '/' + 'main_cls.py.backup')
    os.system('cp model.py outputs' + '/' + exp_name + '/' + 'model.py.backup')
    os.system('cp util.py outputs' + '/' + exp_name + '/' + 'util.py.backup')
    os.system('cp data.py outputs' + '/' + exp_name + '/' + 'data.py.backup')

class IOStream():
    def __init__(self, path):
        self.f = open(path, 'a')

    def cprint(self, text):
        print(text)
        self.f.write(text+'\n')
        self.f.flush()

    def close(self):
        self.f.close()

def train(io):


    batch_size = 4
    test_batch_size = 1
    epochs = 1001
    lr = 2 * 1e-3
    momentum = 0.9
    scheduler = 'cos'
    no_cuda = False

    file_path = "/content/training/images/"
    label_path = "/content/training/1st_manual/"
    #file_path = "/"
    #label_path = "F:/DRIVE/training/1st_manual/"
    # file_path = "E:/vasular/CHASEDB1/images/"
    # label_path = "E:/vasular/CHASEDB1/1st_label/"
    # file_path = "G:/vasular/STAREdatabase/images/"
    # label_path = "G:/vasular/STAREdatabase/labels-ah/"
    # file_path = "G:/vasular/HRFdatas/images/"
    # label_path = "G:/vasular/HRFdatas/images/"
    train_loader = DataLoader(TrainData(file_path, label_path, train_flag = True), num_workers=0,
                              batch_size=batch_size, shuffle=True, drop_last=True)

    file_path = "/content/test/images/"
    label_path = "/content/test/1st_manual/"
    #file_path = "F:/DRIVE/test/images/"
    #label_path = "F:/DRIVE/test/1st_manual/"
    # file_path = "E:/vasular/CHASEDB1/test/"
    # label_path = "E:/vasular/CHASEDB1/1st_label/"
    # file_path = "G:/vasular/STAREdatabase/test/"
    # label_path = "G:/vasular/STAREdatabase/labels-ah/"
    # file_path = "G:/vasular/HRFdatas/test/"
    # label_path = "G:/vasular/HRFdatas/test/"
    test_loader = DataLoader(TestData(file_path, label_path, train_flag = False), num_workers=0,
                             batch_size=test_batch_size, shuffle=True, drop_last=False)

    # Try to load models
    model = RetinalVasularSegFF(in_channels=3,
                            out_channels=2,
                            img_size=[192, 192],
                            feature_size=16,
                            num_heads=4,
                            depths=[3, 3, 3, 3],
                            dims=[32, 64, 128, 256],
                              hidden_size=256,
                            do_ds=True,
                            )

    model_name = "./outputs/teethseg_model_200.pth"
    # model_initial(model, model_name)

    opt = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = CosineAnnealingLR(opt, epochs, eta_min=1e-6, last_epoch = -1)

    focalLoss = FocalLoss2d(gamma=2)#WeightedFocalLoss()#

    model.cuda()
    model.train()
    scaler = GradScaler()
    inter_nums = len(train_loader)
    total_acc = 0
    for epoch in range(0, epochs):
        ####################
        # Train
        ####################

        if scheduler == 'cos':
            scheduler.step()
        elif scheduler == 'step':
            if opt.param_groups[0]['lr'] > 1e-5:
                scheduler.step()
            if opt.param_groups[0]['lr'] < 1e-5:
                for param_group in opt.param_groups:
                    param_group['lr'] = 1e-5

        train_loss = 0.0
        loss_dice = 0
        sen_v = 0
        acc_v = 0
        spec_v = 0
        # for data, edges, label in train_loader:
        tic = time.time()
        nums = 0
        model.train()
        for train_data, train_label, weight_ in train_loader:

            train_data, train_label, train_weight_ = data_crop(train_data, train_label, weight_)

            train_data = train_data.cuda().float()
            train_label = train_label.cuda().float()
            train_weight_ = train_weight_.cuda().float()

            nums = nums +1
            opt.zero_grad()
            with autocast():
                outputs_seg, out2, _ = model(train_data)

                train_label[train_label >= 0.5] = 1
                loss_seg = F.cross_entropy(outputs_seg, train_label.long())
                # loss_seg = F.binary_cross_entropy_with_logits(outputs_seg[:,0, :, :], train_label)
                sec_loss = focalLoss(outputs_seg, train_label[:, :, :].long())

                outputs_soft = torch.softmax(outputs_seg, dim=1)[:, 1, :, :]#F.sigmoid(outputs_seg[:, 0, :, :])#
                loss_dice_ = dice_loss(outputs_soft, train_label[:, :, :])

                sen_v_, acc_v_, spec_v_ = cal_sen(outputs_soft, train_label[:, :, :].long())

                loss = (loss_seg + 1*loss_dice_ + 2*sec_loss)*1  #

            scaler.scale(loss).backward()
            # Unscales gradients and calls
            # or skips optimizer.step()
            scaler.step(opt)
            # Updates the scale for next iteration
            scaler.update()

            train_loss += loss.item()
            loss_dice += loss_dice_.item()
            sen_v += sen_v_.item()
            acc_v += acc_v_.item()
            spec_v += spec_v_.item()
            VIEW_NUMS = 5
            if nums % VIEW_NUMS == 0:
                toc = time.time()
                train_loss = train_loss/ (VIEW_NUMS)
                loss_dice = loss_dice / (VIEW_NUMS)
                sen_v = sen_v/(VIEW_NUMS)
                acc_v = acc_v/(VIEW_NUMS)
                spec_v = spec_v/(VIEW_NUMS)

                print("lr = ", opt.param_groups[0]['lr'])
                outstr = 'epoch %d /%d,epoch %d /%d, loss: %.6f, loss_dice: %.6f, sen_v: %.6f, acc_v: %.6f, spec_v: %.6f, const time: %.6f' % (
                 epoch,epochs, nums, inter_nums, train_loss, loss_dice, sen_v, acc_v, spec_v, toc - tic)

                io.cprint(outstr)
                train_loss = 0.0
                loss_dice = 0
                sen_v = 0
                acc_v = 0
                spec_v = 0
                tic = time.time()
        if 0 == epoch % 10 and epoch>10:
            test_nums = 0
            CROP_SIZE = 192
            loss_dice, sen_v, acc_v, spec_v =0, 0, 0, 0
            model.eval()
            patch_size = [CROP_SIZE, CROP_SIZE]
            stride_y, stride_x = 128, 128
            for test_data, test_label in test_loader:
                test_data = test_data.cuda().float()
                test_label = test_label.cuda().float()
                nums = nums + 1
                test_nums = test_nums + 1
                hh, ww = test_label.shape[-2:]
                with autocast():
                    score_map = model_forward(model, test_data, patch_size, hh, ww, stride_y, stride_x)
                    train_label[train_label > 0.5] = 1
                    loss_dice_ = dice_loss(score_map, test_label.long())
                    sen_v_, acc_v_, spec_v_ = cal_sen(score_map.detach(), test_label.detach().long())
                loss = 1.0 * (loss_dice_).item()
                sen_v = sen_v + sen_v_.item()
                acc_v = acc_v + acc_v_.item()
                spec_v = spec_v + spec_v_.item()
                loss_dice = loss_dice + loss_dice_.item()
            sen_v = sen_v/test_nums
            acc_v = acc_v/test_nums
            spec_v = spec_v/test_nums
            loss_dice = loss_dice/test_nums
            toc = time.time()
            outstr = 'epoch %d, loss: %.6f, loss_dice: %.6f, sen_v: %.6f, acc_v: %.6f, spec_v: %.6f, const time: %.6f' % (
                epoch, loss, loss_dice, sen_v, acc_v, spec_v, toc - tic)
            io.cprint("test   "+outstr)
            if (total_acc < (sen_v*0.35+acc_v*0.35+spec_v*0.35)):
                total_acc = (sen_v * 0.35 + acc_v * 0.35 + spec_v * 0.35)
                torch.save({'model': model.state_dict(), 'epoch': epoch},
                           'outputs/' + str(sen_v)+"_"+ str(acc_v)+ "_"+str(spec_v) + '.pth')
        if (epoch) % 200 == 0:
            torch.save({'model': model.state_dict(), 'epoch': epoch}, 'outputs/teethseg_model_' + str(epoch)+ '.pth')


if __name__ == "__main__":


    torch.backends.cudnn.enabled = True
    # Training settings

    exp_name = 'retinal'
    seed = 1

    _init_(exp_name)

    torch.cuda.manual_seed(seed)
    io = IOStream('outputs/' +exp_name + '/run.log')

    train(io)



lr =  0.002
epoch 0 /1001,epoch 5 /5, loss: 1.171121, loss_dice: 0.646048, sen_v: 0.079011, acc_v: 0.831452, spec_v: 0.926893, const time: 6.246006
lr =  0.002
epoch 1 /1001,epoch 5 /5, loss: 0.955764, loss_dice: 0.554631, sen_v: 0.152047, acc_v: 0.879853, spec_v: 0.973801, const time: 3.913092
lr =  0.002
epoch 2 /1001,epoch 5 /5, loss: 0.685662, loss_dice: 0.377291, sen_v: 0.482829, acc_v: 0.907636, spec_v: 0.960470, const time: 3.618625
lr =  0.002
epoch 3 /1001,epoch 5 /5, loss: 0.616338, loss_dice: 0.321664, sen_v: 0.521745, acc_v: 0.915105, spec_v: 0.965293, const time: 3.662418
lr =  0.002
epoch 4 /1001,epoch 5 /5, loss: 0.595184, loss_dice: 0.307318, sen_v: 0.601379, acc_v: 0.913610, spec_v: 0.952901, const time: 3.904668
lr =  0.002
epoch 5 /1001,epoch 5 /5, loss: 0.510428, loss_dice: 0.264044, sen_v: 0.624022, acc_v: 0.927222, spec_v: 0.966460, const time: 3.753670
lr =  0.002
epoch 6 /1001,epoch 5 /5, loss: 0.485185, loss_dice: 0.248519, sen_v: 0.608940, acc_v: 0.929753, spe

ValueError: could not broadcast input array from shape (2,192,192) into shape (192,192)

##test# This is the result of the first run when I asked why the training was not done by changing "main" and the "main" part was run quickly and the test gave low results. #issue ۳#please help

In [ ]:
from __future__ import print_function
import os
import time
import argparse
import cv2
import math
import torch
import torch.nn as nn
from torch.cuda.amp import autocast, GradScaler
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR
#from data.load_train_data import TrainData
#from data.load_test_data import TestData
from torch.utils.data import DataLoader
import sklearn.metrics as metrics
#from net.retinal_vasuclar_net import RetinalVasularSeg
#from net.unetrpp_ff import RetinalVasularSegFF
#from net.UNetFamily import U_Net, AttU_Net, Dense_Unet
#from net.loss import dice_loss, cal_sen
#from metrics import Evaluate
#import config.config as cfg

#def model_initial(model, model_name):
#    # 加载预训练模型
#    pretrained_dict = torch.load(model_name)["model"]
#    model_dict = model.state_dict()
#    # 1. filter out unnecessary keys
#    # pretrained_dictf = {k.replace('module.', ""): v for k, v in pretrained_dict.items() if k.replace('module.', "") in model_dict}
#    pretrained_dictf = {k: v for k, v in pretrained_dict.items() if k in model_dict}
#   # 2. overwrite entries in the existing state dict
#    model_dict.update(pretrained_dictf)
#    # 3. load the new state dict
#    model.load_state_dict(model_dict)

#    print("over")



def model_forward(model, test_data, patch_size, hh,ww, stride_y, stride_x):

    sy = math.ceil((hh - patch_size[0]) / stride_y) + 1
    sx = math.ceil((ww - patch_size[1]) / stride_x) + 1

    score_map = np.zeros((1, hh, ww)).astype(np.float32)
    cnt = np.zeros((1, hh, ww)).astype(np.int32)
    test_datas = []
    boxes = []
    for y in range(0, sy):
        ys = min(stride_y * y, hh - patch_size[0])
        for x in range(0, sx):
            xs = min(stride_x * x, ww - patch_size[1])

            test_patch = test_data[:, :, ys:ys + patch_size[0], xs:xs + patch_size[1]]
            test_datas.append(test_patch)
            boxes.append([ys, ys + patch_size[0], xs, xs + patch_size[1]])

    bsize = 8
    batch_nums = math.ceil(len(test_datas)/bsize)
    for i in range(batch_nums):
        test_patch = torch.cat(test_datas[i*bsize:(i+1)*bsize], dim=0)
        outputs_segb = model(test_patch)[0]
        outputs_softb = F.sigmoid(outputs_segb)
        # outputs_softb = torch.softmax(outputs_segb, dim=1)[:, 1, :, :]
        predb = torch.squeeze(outputs_softb).detach().cpu().numpy()

        for j in range(predb.shape[0]):
            y1, y2, x1, x2 = boxes[i*bsize +j]
            score_map[0, y1: y2, x1: x2] = score_map[0, y1: y2, x1: x2] + predb[j]

            cnt[0, y1: y2, x1: x2] = cnt[0, y1: y2, x1: x2] + 1

    # cv2.namedWindow("score_map", cv2.WINDOW_NORMAL)
    # cv2.imshow("score_map", score_map[0, :, :])
    #
    # cv2.waitKey(0)
    score_map = torch.tensor(score_map / cnt).cuda().float()

    return  score_map


def test():

    file_path = "/content/test/images/"
    label_path = "/content/test/1st_manual/"
    #file_path = "G:/vasular/DRIVE/test/images/"
    #label_path = "G:/vasular/DRIVE/test/1st_manual/"
    # file_path = "G:/vasular/CHASEDB1/test/"
    # label_path = "G:/vasular/CHASEDB1/1st_label/"
    # file_path = "G:/vasular/STAREdatabase/test/"
    # label_path = "G:/vasular/STAREdatabase/labels-ah/"
    test_loader = DataLoader(TestData(file_path, label_path, train_flag = False), num_workers=0,
                             batch_size=1, shuffle=True, drop_last=False)
    device = torch.device("cuda")
    CROP_SIZE = 192
    # Try to load models
    model = RetinalVasularSegFF(in_channels=3,
                            out_channels=1,
                            img_size=[CROP_SIZE, CROP_SIZE],
                            feature_size=16,
                            num_heads=4,
                            depths=[3, 3, 3, 3],
                            dims=[32, 64, 128, 256],
                              hidden_size=256,
                            do_ds=True,
                            )

    # model = U_Net(img_ch=3, output_ch=2)
    #model_name = "./outputs/teethseg_model_1000.pth"
    #model_name = '/content/outputs/cls_1024/models'
    #model_initial(model, model_name)

    model.cuda()
    model.eval()
    evaluate = Evaluate()

    tic = time.time()
    nums = 0
    CROP_SIZE = 192
    sen_v, acc_v, spec_v =0, 0, 0
    patch_size = [CROP_SIZE, CROP_SIZE]
    stride_y, stride_x = 32, 32
    score_maps, test_labels =[], []
    for test_data, test_label in test_loader:

        with torch.no_grad():
            test_data = test_data.cuda().float()
            test_label = test_label.cuda().float()

            hh, ww = test_label.shape[-2:]
            nums = nums +1

            test_label[test_label > 0.5] = 1
            score_map = model_forward(model, test_data, patch_size, hh, ww, stride_y, stride_x)
            #cal Acc
            sen_v_, acc_v_, spec_v_ = cal_sen(score_map, test_label.long())

            # pred = torch.squeeze(outputs_soft[:, 1, :, :]).cpu().numpy() *255
            # img = torch.squeeze(test_data).permute(1, 2, 0).cpu().numpy()*255

            # cv2.imwrite("./outputs/"+ str(nums) + "_"+str(acc_v_.item()) + "p.png", pred)
            # cv2.imwrite("./outputs/" + str(nums) + "_" + str(acc_v_.item()) + "m.png", img)
            print(nums)
            sen_v = sen_v + sen_v_.item()
            acc_v = acc_v + acc_v_.item()
            spec_v = spec_v + spec_v_.item()
            score_maps.append(torch.squeeze(score_map).cpu().numpy())
            test_labels.append(torch.squeeze(test_label.long()).cpu().numpy())
    score_maps = np.array(score_maps)
    test_labels = np.array(test_labels)
    evaluate.add_batch(test_labels, score_maps)
    result = evaluate.save_all_result()
    print(result)
    print("sen_v = ", sen_v/nums, "  ", "acc_v = ", acc_v/nums, "  ", "spec_v = ", spec_v/nums, "  ")



if __name__ == "__main__":

   test()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
OrderedDict([('AUC_ROC', 0.455738808872651), ('AUC_PR', 0.07543721945004993), ('f1-score', 0.16105150446312566), ('Acc', 0.08757803976239545), ('SE', 1.0), ('SP', 0.0), ('precision', 0.08757803976239545)])
sen_v =  1.0    acc_v =  0.08757804036140442    spec_v =  0.0   
